<a href="https://www.kaggle.com/code/paulopilotti/modelo-sassamaru-br-25?scriptVersionId=250526958" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Rating/Raking ELO dos times

| Time               | Rating Final |
|--------------------|-------------|
| Botafogo-RJ        | 1738        |
| Palmeiras          | 1706        |
| Flamengo           | 1691        |
| Internacional      | 1670        |
| Corinthians        | 1664        |
| Fortaleza          | 1660        |
| Sao Paulo          | 1596        |
| Fluminense         | 1587        |
| Gremio             | 1573        |
| Vasco              | 1568        |
| Vitoria            | 1565        |
| Atletico-MG        | 1564        |
| Portuguesa         | 1548        |
| Bragantino         | 1544        |
| Cruzeiro           | 1543        |
| Bahia              | 1538        |
| Santos             | 1524        |
| Barueri            | 1511        |
| Juventude          | 1503        |
| Sport              | 1496        |
| Athletico-PR       | 1493        |
| Goias              | 1484        |
| Santo Andre        | 1480        |
| Ceara              | 1476        |
| Ponte Preta        | 1463        |
| Figueirense        | 1455        |
| Ipatinga           | 1452        |
| Criciuma           | 1452        |
| Cuiaba             | 1450        |
| Atletico-GO        | 1449        |
| Coritiba           | 1438        |
| Joinville          | 1434        |
| America-MG         | 1431        |
| Guarani            | 1423        |
| CSA                | 1422        |
| Paysandu           | 1422        |
| Brasiliense        | 1419        |
| Sao Caetano        | 1417        |
| Avai               | 1416        |
| Gremio Prudente    | 1414        |
| Santa Cruz         | 1389        |
| Parana             | 1384        |
| Nautico            | 1355        |
| Chapecoense        | 1342        |
| America-RN         | 1326        |


# Erros do modelo

| Tipo de Erro                | Quantidade | Descrição                                                      |
|-----------------------------|------------|----------------------------------------------------------------|
| Zebras Clássicas            | 95         | Modelo previu um vencedor e o outro ganhou                     |
| Empates Inesperados         | 98         | Modelo previu um vencedor e deu empate                         |
| Favoritos Desperdiçados     | 0          | Modelo previu empate e alguém ganhou                           |
| **Total de erros**          | **193**    |                                                                |

---

### Cinco erros mais surpreendentes (zebras)

| #  | Data       | Jogo                        | Placar Real | Previsão do Modelo                                 | Resultado Real         |
|----|------------|-----------------------------|-------------|----------------------------------------------------|-----------------------|
| 1  | 29/10/2023 | Internacional vs Coritiba   | 3x4         | Vitória Internacional (65,6% de chance)            | Vitória Coritiba      |
| 2  | 08/10/2023 | Atletico-MG vs Coritiba     | 1x2         | Vitória Atletico-MG (64,7% de chance)              | Vitória Coritiba      |
| 3  | 10/05/2023 | Internacional vs Athletico-PR | 0x2       | Vitória Internacional (60,8% de chance)            | Vitória Athletico-PR  |
| 4  | 19/08/2023 | Internacional vs Fortaleza  | 0x1         | Vitória Internacional (60,7% de chance)            | Vitória Fortaleza     |
| 5  | 06/12/2023 | Santos vs Fortaleza         | 1x2         | Vitória Santos (60,7% de chance)                   | Vitória Fortaleza     |

In [1]:
!pip install pandas

In [2]:
# Importa a biblioteca pandas, que é essencial para trabalhar com dados
import pandas as pd

# Define o nome do arquivo CSV para ser lido
# Certifique-se de que este arquivo esteja na mesma pasta que o seu script
nome_arquivo = '/kaggle/input/br-data-soccer/campeonato-brasileiro-full.csv'

# Carrega os dados do arquivo CSV para um DataFrame do pandas
# Um DataFrame é como uma tabela ou planilha, ideal para organizar os dados
try:
    df = pd.read_csv(nome_arquivo)
    print("Arquivo CSV carregado com sucesso!")
    # Mostra as 5 primeiras linhas da tabela para termos uma ideia dos dados
    print("Amostra dos dados:")
    print(df.head())
except FileNotFoundError:
    print(f"Erro: O arquivo '{nome_arquivo}' não foi encontrado. Verifique se ele está na pasta correta.")

Arquivo CSV carregado com sucesso!
Amostra dos dados:
   ID  rodata        data   hora       mandante    visitante  \
0   1       1  29/03/2003  16:00        Guarani        Vasco   
1   2       1  29/03/2003  16:00   Athletico-PR       Gremio   
2   3       1  30/03/2003  16:00       Flamengo     Coritiba   
3   4       1  30/03/2003  16:00          Goias     Paysandu   
4   5       1  30/03/2003  16:00  Internacional  Ponte Preta   

  formacao_mandante formacao_visitante tecnico_mandante tecnico_visitante  \
0               NaN                NaN              NaN               NaN   
1               NaN                NaN              NaN               NaN   
2               NaN                NaN              NaN               NaN   
3               NaN                NaN              NaN               NaN   
4               NaN                NaN              NaN               NaN   

       vencedor              arena  mandante_Placar  visitante_Placar  \
0       Guarani     Brinc

- `import pandas as pd`: Este comando importa a biblioteca pandas e nos permite
usar o apelido pd para acessá-la, o que é uma convenção comum.
- `pd.read_csv(...)`: Esta função lê os dados do seu arquivo CSV e os armazena na variável df.
- `df.head()`: Este método nos mostra um "aperitivo" dos nossos dados, exibindo as primeiras cinco partidas da lista.

In [3]:
# Importa a biblioteca pandas, que é essencial para trabalhar com dados
import pandas as pd
import math # Precisaremos da biblioteca de matemática para os cálculos

# --- PARÂMETROS DO MODELO ---
# Pontuação inicial para todos os times quando eles aparecem pela primeira vez
RATING_INICIAL = 1500
# O fator K define o quão rápido os ratings mudam. Valores maiores = mudanças mais rápidas.
K_FACTOR = 30
# Vantagem de jogar em casa: adiciona pontos ao rating do time da casa para o cálculo
VANTAGEM_CASA = 100

# --- FUNÇÕES DO SISTEMA ELO ---

def calcular_expectativa(rating1, rating2):
    """
    Calcula a probabilidade de vitória do time 1 contra o time 2.
    A fórmula é a padrão do sistema Elo.
    """
    return 1 / (1 + 10 ** ((rating2 - rating1) / 400))

def atualizar_ratings(rating_casa, rating_visitante, placar_casa, placar_visitante):
    """
    Atualiza os ratings dos times com base no resultado da partida.
    """
    # Define o resultado real (S) da partida: 1 para vitória, 0.5 para empate, 0 para derrota
    if placar_casa > placar_visitante:
        resultado_real_casa = 1
    elif placar_casa < placar_visitante:
        resultado_real_casa = 0
    else:
        resultado_real_casa = 0.5

    # Para o visitante, o resultado é o inverso
    resultado_real_visitante = 1 - resultado_real_casa

    # Calcula a expectativa de vitória para o time da casa, considerando a vantagem de jogar em casa
    expectativa_casa = calcular_expectativa(rating_casa + VANTAGEM_CASA, rating_visitante)

    # A expectativa do visitante é o inverso
    expectativa_visitante = 1 - expectativa_casa

    # Atualiza o rating do time da casa
    novo_rating_casa = rating_casa + K_FACTOR * (resultado_real_casa - expectativa_casa)

    # Atualiza o rating do time visitante
    novo_rating_visitante = rating_visitante + K_FACTOR * (resultado_real_visitante - expectativa_visitante)

    return novo_rating_casa, novo_rating_visitante


# --- PROCESSAMENTO DOS DADOS ---

# Carrega os dados
try:
    df = pd.read_csv('/kaggle/input/br-data-soccer/campeonato-brasileiro-full.csv')

    # Dicionário para armazenar o rating atual de cada time
    ratings = {}

    # Itera sobre cada linha (partida) do nosso arquivo de dados
    for indice, partida in df.iterrows():
        time_casa = partida['mandante']
        time_visitante = partida['visitante']
        placar_casa = partida['mandante_Placar']
        placar_visitante = partida['visitante_Placar']

        # Se um time ainda não está no nosso dicionário, o adicionamos com o rating inicial
        if time_casa not in ratings:
            ratings[time_casa] = RATING_INICIAL
        if time_visitante not in ratings:
            ratings[time_visitante] = RATING_INICIAL

        # Pega o rating atual dos times antes da partida
        rating_atual_casa = ratings[time_casa]
        rating_atual_visitante = ratings[time_visitante]

        # Calcula os novos ratings após a partida
        novo_rating_casa, novo_rating_visitante = atualizar_ratings(
            rating_atual_casa,
            rating_atual_visitante,
            placar_casa,
            placar_visitante
        )

        # Atualiza o dicionário com os novos ratings
        ratings[time_casa] = novo_rating_casa
        ratings[time_visitante] = novo_rating_visitante

    # Ao final do loop, vamos exibir os ratings finais dos times
    # Ordenamos do maior para o menor para ver o "ranking de força"
    ratings_ordenados = sorted(ratings.items(), key=lambda item: item[1], reverse=True)

    print("--- Ratings Finais dos Times (Força) ---")
    for time, rating in ratings_ordenados:
        print(f"{time}: {int(rating)}")

except FileNotFoundError:
    print("Erro: O arquivo CSV não foi encontrado.")
except KeyError as e:
    print(f"Erro: A coluna {e} não foi encontrada no CSV. Verifique os nomes das colunas.")

--- Ratings Finais dos Times (Força) ---
Botafogo-RJ: 1738
Palmeiras: 1706
Flamengo: 1691
Internacional: 1670
Corinthians: 1664
Fortaleza: 1660
Sao Paulo: 1596
Fluminense: 1587
Gremio: 1573
Vasco: 1568
Vitoria: 1565
Atletico-MG: 1564
Portuguesa: 1548
Bragantino: 1544
Cruzeiro: 1543
Bahia: 1538
Santos: 1524
Barueri: 1511
Juventude: 1503
Sport: 1496
Athletico-PR: 1493
Goias: 1484
Santo Andre: 1480
Ceara: 1476
Ponte Preta: 1463
Figueirense: 1455
Ipatinga: 1452
Criciuma: 1452
Cuiaba: 1450
Atletico-GO: 1449
Coritiba: 1438
Joinville: 1434
America-MG: 1431
Guarani: 1423
CSA: 1422
Paysandu: 1422
Brasiliense: 1419
Sao Caetano: 1417
Avai: 1416
Gremio Prudente: 1414
Santa Cruz: 1389
Parana: 1384
Nautico: 1355
Chapecoense: 1342
America-RN: 1326


- `RATING_INICIAL, K_FACTOR, VANTAGEM_CASA`: São os parâmetros do nosso modelo.
- `ratings = {}`: Criamos um dicionário, que é uma estrutura de dados perfeita para guardar pares de "chave" e "valor" (no nosso caso, "nome do time" e "pontuação").
- `df.iterrows()`: Este é o laço principal. Ele passa por cada jogo, um por um, em ordem cronológica, o que é crucial para que o sistema de rating funcione corretamente.
- `sorted(...)`: Após calcular tudo, usamos esta função para ordenar os times pela pontuação final e exibir um ranking.

In [4]:
# -*- coding: utf-8 -*-
import pandas as pd
import math

# --- PARÂMETROS DO MODELO ---
RATING_INICIAL = 1500
K_FACTOR = 30
VANTAGEM_CASA = 100 # Pontos de rating adicionados ao time da casa

# --- FUNÇÕES DO SISTEMA ELO ---

def calcular_expectativa(rating1, rating2):
    """Calcula a probabilidade de vitória do time 1 contra o time 2."""
    return 1 / (1 + 10 ** ((rating2 - rating1) / 400))

def atualizar_ratings(rating_casa, rating_visitante, placar_casa, placar_visitante):
    """Atualiza os ratings com base no resultado da partida."""
    # Define o resultado real (S): 1 para vitória, 0.5 para empate, 0 para derrota
    if placar_casa > placar_visitante:
        resultado_real_casa = 1.0
    elif placar_casa < placar_visitante:
        resultado_real_casa = 0.0
    else:
        resultado_real_casa = 0.5

    resultado_real_visitante = 1.0 - resultado_real_casa

    # Calcula a expectativa de vitória para o time da casa (com vantagem de casa)
    expectativa_casa = calcular_expectativa(rating_casa + VANTAGEM_CASA, rating_visitante)
    expectativa_visitante = 1 - expectativa_casa # A soma das expectativas é 1

    # Atualiza os ratings usando a fórmula do Elo
    novo_rating_casa = rating_casa + K_FACTOR * (resultado_real_casa - expectativa_casa)
    novo_rating_visitante = rating_visitante + K_FACTOR * (resultado_real_visitante - expectativa_visitante)

    return novo_rating_casa, novo_rating_visitante

# --- FUNÇÃO DE PREVISÃO ---

def prever_partida(time_casa, time_visitante, ratings):
    """
    Prevê o resultado de uma partida com base nos ratings atuais.
    Retorna as probabilidades de (Vitória Casa, Empate, Vitória Visitante).
    """
    if time_casa not in ratings or time_visitante not in ratings:
        print("Um ou ambos os times não foram encontrados nos dados históricos.")
        return

    rating_casa = ratings[time_casa]
    rating_visitante = ratings[time_visitante]

    # Calcula a probabilidade de vitória para cada lado
    prob_vitoria_casa = calcular_expectativa(rating_casa + VANTAGEM_CASA, rating_visitante)
    prob_vitoria_visitante = calcular_expectativa(rating_visitante, rating_casa + VANTAGEM_CASA) # Sem vantagem de casa para o visitante

    # Uma forma simples de estimar o empate é subtrair as vitórias do total (100%)
    # Modelos mais complexos podem usar outras fórmulas aqui
    prob_empate = 1 - prob_vitoria_casa - prob_vitoria_visitante

    # Ajuste para garantir que as probabilidades somem 100% e não sejam negativas
    # (pode ocorrer em casos de grande diferença de rating)
    if prob_empate < 0: prob_empate = 0
    total_prob = prob_vitoria_casa + prob_vitoria_visitante + prob_empate

    prob_vitoria_casa /= total_prob
    prob_vitoria_visitante /= total_prob
    prob_empate /= total_prob

    print("\n--- 🔮 PREVISÃO DA PARTIDA 🔮 ---")
    print(f"Jogo: {time_casa} vs. {time_visitante}")
    print(f"Força (Rating): {int(rating_casa)} vs. {int(rating_visitante)}")
    print("---------------------------------")
    print(f"Chance de Vitória {time_casa}: {prob_vitoria_casa:.1%}")
    print(f"Chance de Empate: {prob_empate:.1%}")
    print(f"Chance de Vitória {time_visitante}: {prob_vitoria_visitante:.1%}")
    print("---------------------------------")

# --- EXECUÇÃO PRINCIPAL ---

def main():
    """Função principal que executa todo o processo."""
    try:
        df = pd.read_csv('/kaggle/input/br-data-soccer/campeonato-brasileiro-full.csv')
    except FileNotFoundError:
        print("ERRO: Arquivo 'campeonato-brasileiro-full.csv' não encontrado.")
        return

    # Limpeza básica: remove linhas onde o placar é inválido (NaN)
    df.dropna(subset=['mandante_Placar', 'visitante_Placar'], inplace=True)

    ratings = {}

    # Itera sobre as partidas para treinar o modelo (calcular os ratings)
    for _, partida in df.iterrows():
        time_casa = partida['mandante']
        time_visitante = partida['visitante']
        placar_casa = int(partida['mandante_Placar'])
        placar_visitante = int(partida['visitante_Placar'])

        if time_casa not in ratings: ratings[time_casa] = RATING_INICIAL
        if time_visitante not in ratings: ratings[time_visitante] = RATING_INICIAL

        rating_atual_casa = ratings[time_casa]
        rating_atual_visitante = ratings[time_visitante]

        novo_rating_casa, novo_rating_visitante = atualizar_ratings(
            rating_atual_casa, rating_atual_visitante, placar_casa, placar_visitante
        )

        ratings[time_casa] = novo_rating_casa
        ratings[time_visitante] = novo_rating_visitante

    # --- FAÇA SUAS PREVISÕES AQUI ---
    # Altere os nomes dos times abaixo para prever diferentes partidas!
    #prever_partida("Flamengo", "Palmeiras", ratings)
    prever_partida("Gremio", "Internacional", ratings)
    prever_partida("Internacional", "Gremio", ratings)


# Garante que o script será executado apenas quando rodado diretamente
if __name__ == "__main__":
    main()


--- 🔮 PREVISÃO DA PARTIDA 🔮 ---
Jogo: Gremio vs. Internacional
Força (Rating): 1573 vs. 1670
---------------------------------
Chance de Vitória Gremio: 50.4%
Chance de Empate: 0.0%
Chance de Vitória Internacional: 49.6%
---------------------------------

--- 🔮 PREVISÃO DA PARTIDA 🔮 ---
Jogo: Internacional vs. Gremio
Força (Rating): 1670 vs. 1573
---------------------------------
Chance de Vitória Internacional: 75.7%
Chance de Empate: 0.0%
Chance de Vitória Gremio: 24.3%
---------------------------------


## O Modelo de Gols (Poisson)

A ideia principal é: bons ataques marcam mais gols, boas defesas sofrem menos gols. Vamos quantificar isso.

- Força de Ataque e Defesa: Em vez de um único rating de "força", cada time terá duas métricas:
- Força de Ataque: Sua capacidade de marcar gols em comparação com a média da liga.
- Força de Defesa: Sua capacidade de evitar gols em comparação com a média da liga.
- Previsão de Gols Esperados: Para um jogo Time A vs Time B, vamos calcular quantos gols esperamos que cada time marque. Isso será uma combinação da Força de Ataque do Time A com a Fraqueza da Defesa do Time B.
- Distribuição de Poisson: Uma vez que temos os "gols esperados" (ex: esperamos que o Time A marque 1.7 gols), a função matemática de Poisson nos dirá a probabilidade de o Time A marcar exatamente 0 gols, 1 gol, 2 gols, e assim por diante.
- Cálculo das Probabilidades Finais: Ao calcular a chance de cada placar possível (0x0, 1x0, 0x1, 1x1, 2x1, etc.), podemos somar todos os resultados que levam a uma vitória do mandante,

In [5]:
# -*- coding: utf-8 -*-
import pandas as pd
import math

MAX_GOLS_CONSIDERADOS = 8

# --- FUNÇÕES DO MODELO ---

def calcular_forcas(df):
    """
    Calcula as forças de ataque e defesa de cada time com base nos dados históricos.
    """
    # Dicionário para guardar as forças de cada time
    forcas = {}

    # Calcula as médias de gols da liga inteira
    media_gols_marcados_casa = df['mandante_Placar'].mean()
    media_gols_sofridos_casa = df['visitante_Placar'].mean()
    media_gols_marcados_fora = df['visitante_Placar'].mean()
    media_gols_sofridos_fora = df['mandante_Placar'].mean()

    # Pega a lista de todos os times únicos no dataset
    times = pd.unique(df[['mandante', 'visitante']].values.ravel('K'))

    for time in times:
        # Filtra os jogos do time como mandante e visitante
        jogos_casa = df[df['mandante'] == time]
        jogos_fora = df[df['visitante'] == time]

        # Calcula a média de gols do time em casa
        media_time_marcados_casa = jogos_casa['mandante_Placar'].mean()
        media_time_sofridos_casa = jogos_casa['visitante_Placar'].mean()

        # Calcula a média de gols do time fora de casa
        media_time_marcados_fora = jogos_fora['visitante_Placar'].mean()
        media_time_sofridos_fora = jogos_fora['mandante_Placar'].mean()

        # Calcula a força de ataque (time marca X% a mais/menos que a média)
        forca_ataque_casa = media_time_marcados_casa / media_gols_marcados_casa
        forca_ataque_fora = media_time_marcados_fora / media_gols_marcados_fora

        # Calcula a força de defesa (time sofre X% a mais/menos que a média)
        # Nota: Uma defesa forte tem um valor baixo (ex: 0.80)
        forca_defesa_casa = media_time_sofridos_casa / media_gols_sofridos_casa
        forca_defesa_fora = media_time_sofridos_fora / media_gols_sofridos_fora

        forcas[time] = {
            'ataque_casa': forca_ataque_casa,
            'defesa_casa': forca_defesa_casa,
            'ataque_fora': forca_ataque_fora,
            'defesa_fora': forca_defesa_fora
        }

    medias_liga = {
        'gols_casa': media_gols_marcados_casa,
        'gols_fora': media_gols_marcados_fora
    }

    return forcas, medias_liga

def poisson(gols_esperados, num_gols_observado):
    """
    Calcula a probabilidade de observar um certo número de gols,
    dado o número de gols esperados. Esta é a fórmula da Distribuição de Poisson.
    """
    return (gols_esperados ** num_gols_observado) * math.exp(-gols_esperados) / math.factorial(num_gols_observado)


def prever_partida_poisson(time_casa, time_visitante, forcas, medias_liga):
    """
    Prevê o resultado de uma partida usando o modelo de Poisson.
    """
    if time_casa not in forcas or time_visitante not in forcas:
        print(f"\nAVISO: Não foi possível prever {time_casa} vs {time_visitante}. Um dos times não tem dados históricos suficientes.")
        return

    # Pega as forças de ataque e defesa dos times
    ataque_casa = forcas[time_casa]['ataque_casa']
    defesa_casa = forcas[time_casa]['defesa_casa']
    ataque_visitante = forcas[time_visitante]['ataque_fora']
    defesa_visitante = forcas[time_visitante]['defesa_fora']

    # Calcula o número de gols esperados para cada time na partida
    gols_esperados_casa = ataque_casa * defesa_visitante * medias_liga['gols_casa']
    gols_esperados_visitante = ataque_visitante * defesa_casa * medias_liga['gols_fora']

    # Variáveis para somar as probabilidades
    prob_vitoria_casa = 0
    prob_empate = 0
    prob_vitoria_visitante = 0

    # Itera sobre todos os placares possíveis (de 0x0 até MAX_GOLS x MAX_GOLS)
    for gols_casa in range(MAX_GOLS_CONSIDERADOS + 1):
        for gols_visitante in range(MAX_GOLS_CONSIDERADOS + 1):
            # Calcula a probabilidade deste placar exato acontecer
            prob_placar = poisson(gols_esperados_casa, gols_casa) * poisson(gols_esperados_visitante, gols_visitante)

            # Soma a probabilidade ao resultado correspondente
            if gols_casa > gols_visitante:
                prob_vitoria_casa += prob_placar
            elif gols_casa == gols_visitante:
                prob_empate += prob_placar
            else:
                prob_vitoria_visitante += prob_placar

    # Normaliza as probabilidades para que a soma seja 100%
    total_prob = prob_vitoria_casa + prob_empate + prob_vitoria_visitante
    prob_vitoria_casa /= total_prob
    prob_empate /= total_prob
    prob_vitoria_visitante /= total_prob

    print("\n--- 🔮 PREVISÃO DA PARTIDA (MODELO POISSON) 🔮 ---")
    print(f"Jogo: {time_casa} vs. {time_visitante}")
    print(f"Gols esperados: {time_casa} {gols_esperados_casa:.2f} vs {gols_esperados_visitante:.2f} {time_visitante}")
    print("---------------------------------")
    print(f"Chance de Vitória {time_casa}: {prob_vitoria_casa:.1%}")
    print(f"Chance de Empate: {prob_empate:.1%}")
    print(f"Chance de Vitória {time_visitante}: {prob_vitoria_visitante:.1%}")
    print("---------------------------------")

# --- EXECUÇÃO PRINCIPAL ---
def main():
    """Função principal que executa todo o processo."""
    try:
        df = pd.read_csv('/kaggle/input/br-data-soccer/campeonato-brasileiro-full.csv')
    except FileNotFoundError:
        print("ERRO: Arquivo 'campeonato-brasileiro-full.csv' não encontrado.")
        return

    # Limpeza básica: remove linhas onde o placar é inválido (NaN) e converte para inteiro
    df.dropna(subset=['mandante_Placar', 'visitante_Placar'], inplace=True)
    df['mandante_Placar'] = df['mandante_Placar'].astype(int)
    df['visitante_Placar'] = df['visitante_Placar'].astype(int)

    # 1. "Treina" o modelo calculando as forças de ataque e defesa de todos os times
    forcas, medias_liga = calcular_forcas(df)

    print("Forças de ataque e defesa calculadas com sucesso para todos os times.")

    # 2. Faz as previsões para as partidas desejadas
    # Altere os nomes dos times abaixo para prever diferentes partidas!
    prever_partida_poisson("Flamengo", "Palmeiras", forcas, medias_liga)
    prever_partida_poisson("Gremio", "Internacional", forcas, medias_liga)
    prever_partida_poisson("Corinthians", "Sao Paulo", forcas, medias_liga)
    prever_partida_poisson("Cuiaba", "Fortaleza", forcas, medias_liga)


# Garante que o script será executado apenas quando rodado diretamente
if __name__ == "__main__":
    main()

Forças de ataque e defesa calculadas com sucesso para todos os times.

--- 🔮 PREVISÃO DA PARTIDA (MODELO POISSON) 🔮 ---
Jogo: Flamengo vs. Palmeiras
Gols esperados: Flamengo 1.37 vs 1.10 Palmeiras
---------------------------------
Chance de Vitória Flamengo: 43.0%
Chance de Empate: 26.9%
Chance de Vitória Palmeiras: 30.1%
---------------------------------

--- 🔮 PREVISÃO DA PARTIDA (MODELO POISSON) 🔮 ---
Jogo: Gremio vs. Internacional
Gols esperados: Gremio 1.47 vs 0.94 Internacional
---------------------------------
Chance de Vitória Gremio: 49.6%
Chance de Empate: 26.4%
Chance de Vitória Internacional: 24.0%
---------------------------------

--- 🔮 PREVISÃO DA PARTIDA (MODELO POISSON) 🔮 ---
Jogo: Corinthians vs. Sao Paulo
Gols esperados: Corinthians 1.28 vs 1.04 Sao Paulo
---------------------------------
Chance de Vitória Corinthians: 42.0%
Chance de Empate: 28.0%
Chance de Vitória Sao Paulo: 30.1%
---------------------------------

--- 🔮 PREVISÃO DA PARTIDA (MODELO POISSON) 🔮 ---
J

## Abordagem 2

- Vantagem de Casa Específica: Em vez de dar os mesmos 100 pontos de bônus para todo time que joga em casa, vamos calcular uma vantagem customizada para cada clube com base no seu histórico.
- K-Factor Dinâmico: Em vez de uma troca de pontos fixa, a quantidade de pontos de rating transferidos em uma vitória vai depender da margem de gols. Vitórias mais elásticas contarão mais.

In [6]:
# -*- coding: utf-8 -*-
import pandas as pd
import math

# --- PARÂMETROS DO MODELO ---
RATING_INICIAL = 1500
K_FACTOR_BASE = 30 # O K-Factor que será ajustado dinamicamente
VANTAGEM_CASA_PADRAO = 80 # Vantagem para times sem dados históricos suficientes

# --- FUNÇÕES DO MODELO ---

def calcular_vantagens_casa(df):
    """
    NOVO: Calcula uma vantagem de casa específica para cada time.
    A vantagem é baseada no saldo de gols médio do time em casa.
    """
    print("Calculando vantagem de casa específica por time...")
    vantagens = {}
    times = pd.unique(df['mandante'])

    for time in times:
        jogos_casa = df[df['mandante'] == time]
        # Calcula o saldo de gols médio do time quando joga em casa
        saldo_de_gols_medio = (jogos_casa['mandante_Placar'] - jogos_casa['visitante_Placar']).mean()

        # Converte o saldo de gols em uma pontuação de rating.
        # A fórmula aqui é empírica, você pode ajustá-la.
        # Um saldo de +1.0 gol vira ~100 pontos de vantagem.
        vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)

        # Garante que a vantagem não seja negativa (caso um time seja muito ruim em casa)
        vantagens[time] = max(0, vantagem_rating)

    print("Cálculo de vantagens de casa concluído.")
    return vantagens

def calcular_expectativa(rating1, rating2):
    """Calcula a probabilidade de vitória do time 1 contra o time 2."""
    return 1 / (1 + 10 ** ((rating2 - rating1) / 400))

def atualizar_ratings(rating_casa, rating_visitante, placar_casa, placar_visitante, vantagem_casa_time):
    """
    MODIFICADO: Atualiza os ratings com K-Factor dinâmico e vantagem de casa específica.
    """
    # Define o resultado real (S): 1 para vitória, 0.5 para empate, 0 para derrota
    if placar_casa > placar_visitante:
        resultado_real_casa = 1.0
        margem_gols = placar_casa - placar_visitante
    elif placar_casa < placar_visitante:
        resultado_real_casa = 0.0
        margem_gols = placar_visitante - placar_casa
    else:
        resultado_real_casa = 0.5
        margem_gols = 0

    # MODIFICAÇÃO: K-Factor Dinâmico
    # O K-Factor aumenta com a margem de gols. O logaritmo suaviza o efeito.
    k_ajustado = K_FACTOR_BASE * math.log(margem_gols + 1)
    if resultado_real_casa == 0.5: # Para empates, usamos o K-Factor base
        k_ajustado = K_FACTOR_BASE

    resultado_real_visitante = 1.0 - resultado_real_casa

    # MODIFICAÇÃO: Usa a vantagem de casa específica do time
    expectativa_casa = calcular_expectativa(rating_casa + vantagem_casa_time, rating_visitante)
    expectativa_visitante = 1 - expectativa_casa

    # Atualiza os ratings usando a fórmula do Elo com o K-Factor ajustado
    novo_rating_casa = rating_casa + k_ajustado * (resultado_real_casa - expectativa_casa)
    novo_rating_visitante = rating_visitante + k_ajustado * (resultado_real_visitante - expectativa_visitante)

    return novo_rating_casa, novo_rating_visitante

def prever_partida(time_casa, time_visitante, ratings, vantagens_casa):
    """
    MODIFICADO: A previsão agora também considera a vantagem de casa específica.
    """
    if time_casa not in ratings or time_visitante not in ratings:
        print("Um ou ambos os times não foram encontrados nos dados históricos.")
        return

    rating_casa = ratings[time_casa]
    rating_visitante = ratings[time_visitante]

    # Pega a vantagem específica do time da casa, ou a padrão se não houver
    vantagem_time = vantagens_casa.get(time_casa, VANTAGEM_CASA_PADRAO)

    # Calcula a probabilidade de vitória para cada lado
    rating_casa_com_vantagem = rating_casa + vantagem_time
    prob_vitoria_casa = calcular_expectativa(rating_casa_com_vantagem, rating_visitante)
    prob_vitoria_visitante = 1 - prob_vitoria_casa # Simplificado para o modelo Elo

    print("\n--- 🔮 PREVISÃO DA PARTIDA (ELO REFINADO) 🔮 ---")
    print(f"Jogo: {time_casa} vs. {time_visitante}")
    print(f"Força (Rating): {int(rating_casa)} vs. {int(rating_visitante)}")
    print(f"Vantagem de Casa para {time_casa}: +{int(vantagem_time)} pts de rating")
    print("---------------------------------")
    # Nota: O modelo Elo puro não calcula empates diretamente, mas podemos estimar
    print(f"Chance de Vitória {time_casa}: {prob_vitoria_casa:.1%}")
    print(f"Chance de Vitória {time_visitante}: {prob_vitoria_visitante:.1%}")
    print("---------------------------------")

# --- EXECUÇÃO PRINCIPAL ---
def main():
    try:
        df = pd.read_csv('/kaggle/input/br-data-soccer/campeonato-brasileiro-full.csv')
    except FileNotFoundError:
        print("ERRO: Arquivo 'campeonato-brasileiro-full.csv' não encontrado.")
        return

    df.dropna(subset=['mandante_Placar', 'visitante_Placar'], inplace=True)
    df['mandante_Placar'] = df['mandante_Placar'].astype(int)
    df['visitante_Placar'] = df['visitante_Placar'].astype(int)

    # PASSO NOVO: Calcula as vantagens de casa antes de tudo
    vantagens_casa = calcular_vantagens_casa(df)

    ratings = {}

    for _, partida in df.iterrows():
        time_casa = partida['mandante']
        time_visitante = partida['visitante']
        placar_casa = partida['mandante_Placar']
        placar_visitante = partida['visitante_Placar']

        if time_casa not in ratings: ratings[time_casa] = RATING_INICIAL
        if time_visitante not in ratings: ratings[time_visitante] = RATING_INICIAL

        rating_atual_casa = ratings[time_casa]
        rating_atual_visitante = ratings[time_visitante]

        # Pega a vantagem específica do time, ou a padrão se não existir
        vantagem_especifica = vantagens_casa.get(time_casa, VANTAGEM_CASA_PADRAO)

        novo_rating_casa, novo_rating_visitante = atualizar_ratings(
            rating_atual_casa, rating_atual_visitante, placar_casa, placar_visitante, vantagem_especifica
        )

        ratings[time_casa] = novo_rating_casa
        ratings[time_visitante] = novo_rating_visitante

    # --- FAÇA SUAS PREVISÕES AQUI ---
    prever_partida("Flamengo", "Palmeiras", ratings, vantagens_casa)
    prever_partida("Gremio", "Internacional", ratings, vantagens_casa)
    prever_partida("Athletico Paranaense", "Corinthians", ratings, vantagens_casa)

if __name__ == "__main__":
    main()

Calculando vantagem de casa específica por time...
Cálculo de vantagens de casa concluído.


/tmp/ipykernel_35/637990535.py:29: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/637990535.py:29: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/637990535.py:29: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/637990535.py:29: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/637990535.py:29: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)



--- 🔮 PREVISÃO DA PARTIDA (ELO REFINADO) 🔮 ---
Jogo: Flamengo vs. Palmeiras
Força (Rating): 1671 vs. 1691
Vantagem de Casa para Flamengo: +45 pts de rating
---------------------------------
Chance de Vitória Flamengo: 53.8%
Chance de Vitória Palmeiras: 46.2%
---------------------------------

--- 🔮 PREVISÃO DA PARTIDA (ELO REFINADO) 🔮 ---
Jogo: Gremio vs. Internacional
Força (Rating): 1554 vs. 1659
Vantagem de Casa para Gremio: +50 pts de rating
---------------------------------
Chance de Vitória Gremio: 42.3%
Chance de Vitória Internacional: 57.7%
---------------------------------
Um ou ambos os times não foram encontrados nos dados históricos.


## Função calcular_vantagens_casa(df):

- O que faz: Antes de começar a simulação jogo a jogo, esta nova função analisa todo o histórico. Para cada time, ela calcula o saldo de gols médio que ele tem quando joga em casa.
- Por que: Times que historicamente goleiam mais em seus estádios (maior saldo de gols) recebem um bônus de rating maior. Isso reflete melhor a realidade de "caldeirões" como a Arena da Baixada ou o Maracanã lotado. A fórmula 60 * (saldo_de_gols_medio ** 0.8) é uma maneira de converter esse saldo em pontos de rating de forma não-linear, mas você pode experimentar com outras!

## Função atualizar_ratings(...):

- O que faz: Agora ela recebe a vantagem_casa_time e também calcula a margem_gols. O K_FACTOR_BASE é multiplicado pelo logaritmo da margem de gols.
- Por que: O log(margem_gols + 1) faz com que o K-Factor aumente com a margem, mas de forma controlada. A diferença entre uma vitória de 1x0 e 2x0 tem mais impacto no K-Factor do que a diferença entre uma vitória de 6x0 e 7x0. Isso evita que goleadas raras distorçam demais os ratings. E, claro, a função agora usa a vantagem específica do time (+85, +110, etc.) em vez de um valor fixo.

## Função prever_partida(...):

 - O que faz: Foi atualizada para também usar o dicionário vantagens_casa e exibir qual foi o bônus aplicado na previsão.
 - Por que: Transparência. Fica claro para o usuário quanto o modelo acredita que o mando de campo daquele time específico vale em pontos de rating.

## Modelo hibrido (ensemble)

In [7]:
# -*- coding: utf-8 -*-
import pandas as pd
import math

# --- PARÂMETROS GERAIS ---
ELO_RATING_INICIAL = 1500
ELO_K_FACTOR_BASE = 30
ELO_VANTAGEM_CASA_PADRAO = 80
POISSON_MAX_GOLS = 8
ELO_INFLUENCE = 0.35

# --- FUNÇÕES DO MODELO ELO ---

def calcular_vantagens_casa(df):
    vantagens = {}
    times = pd.unique(df['mandante'])
    for time in times:
        jogos_casa = df[df['mandante'] == time]
        # CORREÇÃO APLICADA AQUI
        saldo_de_gols_medio = (jogos_casa['mandante_Placar'] - jogos_casa['visitante_Placar']).mean()

        vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
        vantagens[time] = max(0, vantagem_rating)
    return vantagens

def atualizar_ratings_elo(rating_casa, rating_visitante, placar_casa, placar_visitante, vantagem_casa_time):
    if placar_casa > placar_visitante:
        resultado_real_casa, margem_gols = 1.0, placar_casa - placar_visitante
    elif placar_casa < placar_visitante:
        resultado_real_casa, margem_gols = 0.0, placar_visitante - placar_casa
    else:
        resultado_real_casa, margem_gols = 0.5, 0

    k_ajustado = ELO_K_FACTOR_BASE * math.log(margem_gols + 1)
    if resultado_real_casa == 0.5: k_ajustado = ELO_K_FACTOR_BASE

    expectativa_casa = 1 / (1 + 10 ** ((rating_visitante - (rating_casa + vantagem_casa_time)) / 400))

    novo_rating_casa = rating_casa + k_ajustado * (resultado_real_casa - expectativa_casa)
    novo_rating_visitante = rating_visitante + k_ajustado * ((1.0 - resultado_real_casa) - (1 - expectativa_casa))
    return novo_rating_casa, novo_rating_visitante

# --- FUNÇÕES DO MODELO POISSON ---

def calcular_forcas_poisson(df):
    forcas = {}
    # CORREÇÃO APLICADA AQUI
    media_gols_marcados_casa = df['mandante_Placar'].mean()
    media_gols_sofridos_casa = df['visitante_Placar'].mean()
    media_gols_marcados_fora = df['visitante_Placar'].mean()
    media_gols_sofridos_fora = df['mandante_Placar'].mean()
    times = pd.unique(df[['mandante', 'visitante']].values.ravel('K'))

    for time in times:
        forcas[time] = {
            # CORREÇÃO APLICADA AQUI
            'ataque_casa': (df[df['mandante'] == time]['mandante_Placar'].mean() / media_gols_marcados_casa),
            'defesa_casa': (df[df['mandante'] == time]['visitante_Placar'].mean() / media_gols_sofridos_casa),
            'ataque_fora': (df[df['visitante'] == time]['visitante_Placar'].mean() / media_gols_marcados_fora),
            'defesa_fora': (df[df['visitante'] == time]['mandante_Placar'].mean() / media_gols_sofridos_fora)
        }
    medias_liga = {'gols_casa': media_gols_marcados_casa, 'gols_fora': media_gols_marcados_fora}
    return forcas, medias_liga

def poisson(gols_esperados, num_gols):
    return (gols_esperados ** num_gols) * math.exp(-gols_esperados) / math.factorial(num_gols)

# --- FUNÇÃO DE PREVISÃO HÍBRIDA ---

def prever_partida_hibrido(time_casa, time_visitante, elo_ratings, forcas_poisson, medias_liga, vantagens_casa):
    if time_casa not in forcas_poisson or time_visitante not in forcas_poisson:
        print(f"\nAVISO: Não foi possível prever {time_casa} vs {time_visitante}. Faltam dados.")
        return

    ataque_casa = forcas_poisson[time_casa]['ataque_casa']
    defesa_visitante = forcas_poisson[time_visitante]['defesa_fora']
    gols_base_casa = ataque_casa * defesa_visitante * medias_liga['gols_casa']

    ataque_visitante = forcas_poisson[time_visitante]['ataque_fora']
    defesa_casa = forcas_poisson[time_casa]['defesa_casa']
    gols_base_visitante = ataque_visitante * defesa_casa * medias_liga['gols_fora']

    rating_casa = elo_ratings.get(time_casa, ELO_RATING_INICIAL)
    rating_visitante = elo_ratings.get(time_visitante, ELO_RATING_INICIAL)
    vantagem_time_casa = vantagens_casa.get(time_casa, ELO_VANTAGEM_CASA_PADRAO)

    elo_diff = (rating_casa + vantagem_time_casa) - rating_visitante
    fator_ajuste_casa = 1 + (elo_diff / 1000) * ELO_INFLUENCE
    fator_ajuste_visitante = 1 - (elo_diff / 1000) * ELO_INFLUENCE

    gols_finais_casa = gols_base_casa * max(0.1, fator_ajuste_casa)
    gols_finais_visitante = gols_base_visitante * max(0.1, fator_ajuste_visitante)

    prob_vitoria_casa, prob_empate, prob_vitoria_visitante = 0, 0, 0
    for g_c in range(POISSON_MAX_GOLS + 1):
        for g_v in range(POISSON_MAX_GOLS + 1):
            prob_placar = poisson(gols_finais_casa, g_c) * poisson(gols_finais_visitante, g_v)
            if g_c > g_v: prob_vitoria_casa += prob_placar
            elif g_c == g_v: prob_empate += prob_placar
            else: prob_vitoria_visitante += prob_placar

    total_prob = prob_vitoria_casa + prob_empate + prob_vitoria_visitante

    print("\n--- 🔮 PREVISÃO DA PARTIDA (MODELO HÍBRIDO) 🔮 ---")
    print(f"Jogo: {time_casa} vs. {time_visitante}")
    print(f"Rating Elo: {int(rating_casa)} vs {int(rating_visitante)} (Fase Atual)")
    print(f"Gols Esperados (Base Poisson): {gols_base_casa:.2f} vs {gols_base_visitante:.2f}")
    print(f"Gols Esperados (Final Híbrido): {gols_finais_casa:.2f} vs {gols_finais_visitante:.2f}")
    print("---------------------------------")
    print(f"Chance de Vitória {time_casa}: {prob_vitoria_casa / total_prob:.1%}")
    print(f"Chance de Empate: {prob_empate / total_prob:.1%}")
    print(f"Chance de Vitória {time_visitante}: {prob_vitoria_visitante / total_prob:.1%}")
    print("---------------------------------")

# --- EXECUÇÃO PRINCIPAL ---
def main():
    try:
        df = pd.read_csv('/kaggle/input/br-data-soccer/campeonato-brasileiro-full.csv')
    except FileNotFoundError:
        print("ERRO: Arquivo 'campeonato-brasileiro-full.csv' não encontrado.")
        return

    # CORREÇÃO APLICADA AQUI
    colunas_placar = ['mandante_Placar', 'visitante_Placar']
    df.dropna(subset=colunas_placar, inplace=True)
    df[colunas_placar] = df[colunas_placar].astype(int)

    print("Calculando forças estáticas (Poisson e Vantagem de Casa)...")
    forcas_poisson, medias_liga = calcular_forcas_poisson(df)
    vantagens_casa = calcular_vantagens_casa(df)
    print("Cálculos estáticos concluídos.")

    print("\nCalculando ratings dinâmicos (Elo) ao longo da história...")
    elo_ratings = {}
    for _, partida in df.iterrows():
        time_c, time_v = partida['mandante'], partida['visitante']
        # CORREÇÃO APLICADA AQUI
        placar_c, placar_v = partida['mandante_Placar'], partida['visitante_Placar']

        rating_c = elo_ratings.get(time_c, ELO_RATING_INICIAL)
        rating_v = elo_ratings.get(time_v, ELO_RATING_INICIAL)
        vantagem_c = vantagens_casa.get(time_c, ELO_VANTAGEM_CASA_PADRAO)

        novo_rating_c, novo_rating_v = atualizar_ratings_elo(rating_c, rating_v, placar_c, placar_v, vantagem_c)
        elo_ratings[time_c], elo_ratings[time_v] = novo_rating_c, novo_rating_v
    print("Cálculos dinâmicos concluídos.")

    prever_partida_hibrido("Flamengo", "Palmeiras", elo_ratings, forcas_poisson, medias_liga, vantagens_casa)
    prever_partida_hibrido("Gremio", "Internacional", elo_ratings, forcas_poisson, medias_liga, vantagens_casa)
    prever_partida_hibrido("Internacional", "Gremio", elo_ratings, forcas_poisson, medias_liga, vantagens_casa)
    prever_partida_hibrido("Botafogo", "Fluminense", elo_ratings, forcas_poisson, medias_liga, vantagens_casa)

if __name__ == "__main__":
    main()

Calculando forças estáticas (Poisson e Vantagem de Casa)...


/tmp/ipykernel_35/710993689.py:22: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/710993689.py:22: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/710993689.py:22: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/710993689.py:22: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/710993689.py:22: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)


Cálculos estáticos concluídos.

Calculando ratings dinâmicos (Elo) ao longo da história...
Cálculos dinâmicos concluídos.

--- 🔮 PREVISÃO DA PARTIDA (MODELO HÍBRIDO) 🔮 ---
Jogo: Flamengo vs. Palmeiras
Rating Elo: 1671 vs 1691 (Fase Atual)
Gols Esperados (Base Poisson): 1.37 vs 1.10
Gols Esperados (Final Híbrido): 1.38 vs 1.09
---------------------------------
Chance de Vitória Flamengo: 43.6%
Chance de Empate: 26.9%
Chance de Vitória Palmeiras: 29.6%
---------------------------------

--- 🔮 PREVISÃO DA PARTIDA (MODELO HÍBRIDO) 🔮 ---
Jogo: Gremio vs. Internacional
Rating Elo: 1554 vs 1659 (Fase Atual)
Gols Esperados (Base Poisson): 1.47 vs 0.94
Gols Esperados (Final Híbrido): 1.44 vs 0.95
---------------------------------
Chance de Vitória Gremio: 48.4%
Chance de Empate: 26.7%
Chance de Vitória Internacional: 25.0%
---------------------------------

--- 🔮 PREVISÃO DA PARTIDA (MODELO HÍBRIDO) 🔮 ---
Jogo: Internacional vs. Gremio
Rating Elo: 1659 vs 1554 (Fase Atual)
Gols Esperados (Base 

Como saber se nossas mudanças estão realmente melhorando as previsões? Precisamos testá-las.

**O que podemos fazer?** Dividir nossos dados. Por exemplo, usamos os jogos de 2003 a 2022 para treinar o modelo (calcular os ratings) e depois usamos o modelo para prever os jogos de 2023. Como já sabemos os resultados de 2023, podemos comparar as previsões com a realidade e medir a taxa de acerto.

## Modelo Híbrido com Framework de Backtesting

In [8]:
# -*- coding: utf-8 -*-
import pandas as pd
import math

# --- PARÂMETROS GERAIS ---
ELO_RATING_INICIAL = 1500
ELO_K_FACTOR_BASE = 30
ELO_VANTAGEM_CASA_PADRAO = 80
POISSON_MAX_GOLS = 8
ELO_INFLUENCE = 0.2
ANO_DE_TESTE = 2023 # Define o ano que usaremos para testar o modelo

# --- FUNÇÕES DO MODELO (Sem alterações na lógica interna) ---

def calcular_vantagens_casa(df):
    vantagens = {}
    times = pd.unique(df['mandante'])
    for time in times:
        jogos_casa = df[df['mandante'] == time]
        saldo_de_gols_medio = (jogos_casa['mandante_Placar'] - jogos_casa['visitante_Placar']).mean()
        vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
        vantagens[time] = max(0, vantagem_rating)
    return vantagens

def atualizar_ratings_elo(rating_casa, rating_visitante, placar_casa, placar_visitante, vantagem_casa_time):
    if placar_casa > placar_visitante:
        resultado_real_casa, margem_gols = 1.0, placar_casa - placar_visitante
    elif placar_casa < placar_visitante:
        resultado_real_casa, margem_gols = 0.0, placar_visitante - placar_casa
    else:
        resultado_real_casa, margem_gols = 0.5, 0

    k_ajustado = ELO_K_FACTOR_BASE * math.log(margem_gols + 1)
    if resultado_real_casa == 0.5: k_ajustado = ELO_K_FACTOR_BASE
    expectativa_casa = 1 / (1 + 10 ** ((rating_visitante - (rating_casa + vantagem_casa_time)) / 400))
    novo_rating_casa = rating_casa + k_ajustado * (resultado_real_casa - expectativa_casa)
    novo_rating_visitante = rating_visitante + k_ajustado * ((1.0 - resultado_real_casa) - (1 - expectativa_casa))
    return novo_rating_casa, novo_rating_visitante

def calcular_forcas_poisson(df):
    forcas = {}
    media_gols_marcados_casa = df['mandante_Placar'].mean()
    media_gols_sofridos_casa = df['visitante_Placar'].mean()
    media_gols_marcados_fora = df['visitante_Placar'].mean()
    media_gols_sofridos_fora = df['mandante_Placar'].mean()
    times = pd.unique(df[['mandante', 'visitante']].values.ravel('K'))
    for time in times:
        forcas[time] = {
            'ataque_casa': (df[df['mandante'] == time]['mandante_Placar'].mean() / media_gols_marcados_casa),
            'defesa_casa': (df[df['mandante'] == time]['visitante_Placar'].mean() / media_gols_sofridos_casa),
            'ataque_fora': (df[df['visitante'] == time]['visitante_Placar'].mean() / media_gols_marcados_fora),
            'defesa_fora': (df[df['visitante'] == time]['mandante_Placar'].mean() / media_gols_sofridos_fora)
        }
    medias_liga = {'gols_casa': media_gols_marcados_casa, 'gols_fora': media_gols_marcados_fora}
    return forcas, medias_liga

def poisson(gols_esperados, num_gols):
    return (gols_esperados ** num_gols) * math.exp(-gols_esperados) / math.factorial(num_gols)

# --- FUNÇÃO DE PREVISÃO HÍBRIDA (MODIFICADA PARA RETORNAR VALORES) ---

def prever_partida_hibrido(time_casa, time_visitante, elo_ratings, forcas_poisson, medias_liga, vantagens_casa):
    """
    MODIFICADO: Agora retorna as probabilidades em vez de imprimir na tela.
    """
    if time_casa not in forcas_poisson or time_visitante not in forcas_poisson:
        return 0.33, 0.33, 0.34 # Retorna chance igual se faltar dados

    # Lógica de cálculo idêntica à anterior
    ataque_casa = forcas_poisson[time_casa]['ataque_casa']
    defesa_visitante = forcas_poisson[time_visitante]['defesa_fora']
    gols_base_casa = ataque_casa * defesa_visitante * medias_liga['gols_casa']
    ataque_visitante = forcas_poisson[time_visitante]['ataque_fora']
    defesa_casa = forcas_poisson[time_casa]['defesa_casa']
    gols_base_visitante = ataque_visitante * defesa_casa * medias_liga['gols_fora']
    rating_casa = elo_ratings.get(time_casa, ELO_RATING_INICIAL)
    rating_visitante = elo_ratings.get(time_visitante, ELO_RATING_INICIAL)
    vantagem_time_casa = vantagens_casa.get(time_casa, ELO_VANTAGEM_CASA_PADRAO)
    elo_diff = (rating_casa + vantagem_time_casa) - rating_visitante
    fator_ajuste_casa = 1 + (elo_diff / 1000) * ELO_INFLUENCE
    fator_ajuste_visitante = 1 - (elo_diff / 1000) * ELO_INFLUENCE
    gols_finais_casa = gols_base_casa * max(0.1, fator_ajuste_casa)
    gols_finais_visitante = gols_base_visitante * max(0.1, fator_ajuste_visitante)
    prob_vitoria_casa, prob_empate, prob_vitoria_visitante = 0, 0, 0
    for g_c in range(POISSON_MAX_GOLS + 1):
        for g_v in range(POISSON_MAX_GOLS + 1):
            prob_placar = poisson(gols_finais_casa, g_c) * poisson(gols_finais_visitante, g_v)
            if g_c > g_v: prob_vitoria_casa += prob_placar
            elif g_c == g_v: prob_empate += prob_placar
            else: prob_vitoria_visitante += prob_placar
    total_prob = prob_vitoria_casa + prob_empate + prob_vitoria_visitante

    # MODIFICAÇÃO: Retorna as probabilidades calculadas
    return prob_vitoria_casa / total_prob, prob_empate / total_prob, prob_vitoria_visitante / total_prob

# --- EXECUÇÃO PRINCIPAL (ORQUESTRADOR DO BACKTESTING) ---
def main():
    try:
        df = pd.read_csv('/kaggle/input/br-data-soccer/campeonato-brasileiro-full.csv')
    except FileNotFoundError:
        print("ERRO: Arquivo 'campeonato-brasileiro-full.csv' não encontrado.")
        return

    df.dropna(subset=['mandante_Placar', 'visitante_Placar', 'data'], inplace=True)
    df['mandante_Placar'] = df['mandante_Placar'].astype(int)
    df['visitante_Placar'] = df['visitante_Placar'].astype(int)
    # Converte a coluna de data para um formato que o Python entende
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

    # 1. DIVISÃO DOS DADOS em treino e teste
    ANO_INICIAL_TREINO = ANO_DE_TESTE - 5
    df_treino = df[(df['data'].dt.year < ANO_DE_TESTE) & (df['data'].dt.year >= ANO_INICIAL_TREINO)]
    df_teste = df[df['data'].dt.year == ANO_DE_TESTE].sort_values(by='data') # Garante a ordem cronológica

    if df_teste.empty:
        print(f"ERRO: Nenhum dado encontrado para o ano de teste ({ANO_DE_TESTE}). Verifique seu arquivo CSV.")
        return

    print(f"Dados divididos: {len(df_treino)} jogos para treino, {len(df_teste)} jogos para teste.")

    # 2. TREINAMENTO com os dados de treino
    print("\nIniciando treinamento com dados até", ANO_DE_TESTE - 1)
    forcas_poisson, medias_liga = calcular_forcas_poisson(df_treino)
    vantagens_casa = calcular_vantagens_casa(df_treino)

    elo_ratings = {}
    for _, partida in df_treino.iterrows():
        time_c, time_v = partida['mandante'], partida['visitante']
        placar_c, placar_v = partida['mandante_Placar'], partida['visitante_Placar']
        rating_c = elo_ratings.get(time_c, ELO_RATING_INICIAL)
        rating_v = elo_ratings.get(time_v, ELO_RATING_INICIAL)
        vantagem_c = vantagens_casa.get(time_c, ELO_VANTAGEM_CASA_PADRAO)
        novo_rating_c, novo_rating_v = atualizar_ratings_elo(rating_c, rating_v, placar_c, placar_v, vantagem_c)
        elo_ratings[time_c], elo_ratings[time_v] = novo_rating_c, novo_rating_v
    print("Treinamento concluído. Os ratings Elo estão prontos para o início da temporada de teste.")

    # 3. TESTE E VALIDAÇÃO na temporada de teste
    print(f"\nIniciando simulação de backtesting para a temporada de {ANO_DE_TESTE}...")
    acertos = 0
    jogos_testados = 0

    for _, partida_teste in df_teste.iterrows():
        jogos_testados += 1
        time_c = partida_teste['mandante']
        time_v = partida_teste['visitante']
        placar_c_real = partida_teste['mandante_Placar']
        placar_v_real = partida_teste['visitante_Placar']

        # Faz a previsão para o jogo atual
        prob_vc, prob_e, prob_vv = prever_partida_hibrido(time_c, time_v, elo_ratings, forcas_poisson, medias_liga, vantagens_casa)

        # Determina a previsão do modelo (o resultado com maior probabilidade)
        previsao = ''
        if prob_vc > prob_e and prob_vc > prob_vv:
            previsao = 'VITORIA_CASA'
        elif prob_vv > prob_vc and prob_vv > prob_e:
            previsao = 'VITORIA_VISITANTE'
        else:
            previsao = 'EMPATE'

        # Determina o resultado real
        resultado_real = ''
        if placar_c_real > placar_v_real:
            resultado_real = 'VITORIA_CASA'
        elif placar_v_real > placar_c_real:
            resultado_real = 'VITORIA_VISITANTE'
        else:
            resultado_real = 'EMPATE'

        # Verifica se o modelo acertou
        if previsao == resultado_real:
            acertos += 1

        # ATUALIZAÇÃO CRUCIAL: Atualiza os ratings Elo com o resultado do jogo que acabou de acontecer
        rating_c = elo_ratings.get(time_c, ELO_RATING_INICIAL)
        rating_v = elo_ratings.get(time_v, ELO_RATING_INICIAL)
        vantagem_c = vantagens_casa.get(time_c, ELO_VANTAGEM_CASA_PADRAO)
        novo_rating_c, novo_rating_v = atualizar_ratings_elo(rating_c, rating_v, placar_c_real, placar_v_real, vantagem_c)
        elo_ratings[time_c], elo_ratings[time_v] = novo_rating_c, novo_rating_v

    # 4. RESULTADO FINAL DO BACKTESTING
    acuracia = (acertos / jogos_testados) * 100
    print("\n--- ✅ RESULTADO DO BACKTESTING ✅ ---")
    print(f"Temporada testada: {ANO_DE_TESTE}")
    print(f"Total de jogos testados: {jogos_testados}")
    print(f"Acertos do modelo: {acertos}")
    print(f"ACURÁCIA FINAL: {acuracia:.2f}%")
    print("---------------------------------------")


if __name__ == "__main__":
    main()

Dados divididos: 1900 jogos para treino, 380 jogos para teste.

Iniciando treinamento com dados até 2022
Treinamento concluído. Os ratings Elo estão prontos para o início da temporada de teste.

Iniciando simulação de backtesting para a temporada de 2023...

--- ✅ RESULTADO DO BACKTESTING ✅ ---
Temporada testada: 2023
Total de jogos testados: 380
Acertos do modelo: 175
ACURÁCIA FINAL: 46.05%
---------------------------------------


/tmp/ipykernel_35/3654686249.py:21: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/3654686249.py:21: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/3654686249.py:21: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/3654686249.py:21: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/3654686249.py:21: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/3654686249.py:21: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/3654686249.py:21: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating 

In [9]:
# -*- coding: utf-8 -*-
import pandas as pd
import math

# --- PARÂMETROS GERAIS (USANDO NOSSOS VALORES OTIMIZADOS) ---
ELO_RATING_INICIAL = 1500
ELO_K_FACTOR_BASE = 30
ELO_VANTAGEM_CASA_PADRAO = 80
POISSON_MAX_GOLS = 8
ELO_INFLUENCE = 0.35
ANO_DE_TESTE = 2023

def calcular_vantagens_casa(df):
    vantagens = {}
    times = pd.unique(df['mandante'])
    for time in times:
        jogos_casa = df[df['mandante'] == time]
        saldo_de_gols_medio = (jogos_casa['mandante_Placar'] - jogos_casa['visitante_Placar']).mean()
        vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
        vantagens[time] = max(0, vantagem_rating)
    return vantagens

def atualizar_ratings_elo(rating_casa, rating_visitante, placar_casa, placar_visitante, vantagem_casa_time):
    if placar_casa > placar_visitante:
        resultado_real_casa, margem_gols = 1.0, placar_casa - placar_visitante
    elif placar_casa < placar_visitante:
        resultado_real_casa, margem_gols = 0.0, placar_visitante - placar_casa
    else:
        resultado_real_casa, margem_gols = 0.5, 0

    k_ajustado = ELO_K_FACTOR_BASE * math.log(margem_gols + 1)
    if resultado_real_casa == 0.5: k_ajustado = ELO_K_FACTOR_BASE
    expectativa_casa = 1 / (1 + 10 ** ((rating_visitante - (rating_casa + vantagem_casa_time)) / 400))
    novo_rating_casa = rating_casa + k_ajustado * (resultado_real_casa - expectativa_casa)
    novo_rating_visitante = rating_visitante + k_ajustado * ((1.0 - resultado_real_casa) - (1 - expectativa_casa))
    return novo_rating_casa, novo_rating_visitante

def calcular_forcas_poisson(df):
    forcas = {}
    media_gols_marcados_casa = df['mandante_Placar'].mean()
    media_gols_sofridos_casa = df['visitante_Placar'].mean()
    media_gols_marcados_fora = df['visitante_Placar'].mean()
    media_gols_sofridos_fora = df['mandante_Placar'].mean()
    times = pd.unique(df[['mandante', 'visitante']].values.ravel('K'))
    for time in times:
        # Usamos .get(0) para evitar erros com times que só jogaram fora ou em casa (raro, mas possível)
        forcas[time] = {
            'ataque_casa': df[df['mandante'] == time]['mandante_Placar'].mean() / media_gols_marcados_casa,
            'defesa_casa': df[df['mandante'] == time]['visitante_Placar'].mean() / media_gols_sofridos_casa,
            'ataque_fora': df[df['visitante'] == time]['visitante_Placar'].mean() / media_gols_marcados_fora,
            'defesa_fora': df[df['visitante'] == time]['mandante_Placar'].mean() / media_gols_sofridos_fora
        }
    medias_liga = {'gols_casa': media_gols_marcados_casa, 'gols_fora': media_gols_marcados_fora}
    return forcas, medias_liga

def poisson(gols_esperados, num_gols):
    if gols_esperados < 0: return 0 # Gols esperados não pode ser negativo
    # Adicionado tratamento para num_gols grande para evitar OverflowError no math.factorial
    if num_gols > 30: return 0
    return (gols_esperados ** num_gols) * math.exp(-gols_esperados) / math.factorial(num_gols)

def prever_partida_hibrido(time_casa, time_visitante, elo_ratings, forcas_poisson, medias_liga, vantagens_casa):
    """
    MODIFICADO: Agora retorna as probabilidades em vez de imprimir na tela.
    """
    # CORREÇÃO APLICADA AQUI: trocado 'visitante' por 'time_visitante'
    if time_casa not in forcas_poisson or time_visitante not in forcas_poisson or \
       pd.isna(forcas_poisson[time_casa]['ataque_casa']) or pd.isna(forcas_poisson[time_visitante]['ataque_fora']):
        return 0.33, 0.33, 0.34

    # Lógica de cálculo idêntica à anterior
    ataque_casa = forcas_poisson[time_casa]['ataque_casa']
    defesa_visitante = forcas_poisson[time_visitante]['defesa_fora']
    gols_base_casa = ataque_casa * defesa_visitante * medias_liga['gols_casa']

    ataque_visitante = forcas_poisson[time_visitante]['ataque_fora']
    defesa_casa = forcas_poisson[time_casa]['defesa_casa']
    gols_base_visitante = ataque_visitante * defesa_casa * medias_liga['gols_fora']

    rating_casa = elo_ratings.get(time_casa, ELO_RATING_INICIAL)
    rating_visitante = elo_ratings.get(time_visitante, ELO_RATING_INICIAL)
    vantagem_time_casa = vantagens_casa.get(time_casa, ELO_VANTAGEM_CASA_PADRAO)

    elo_diff = (rating_casa + vantagem_time_casa) - rating_visitante
    fator_ajuste_casa = 1 + (elo_diff / 1000) * ELO_INFLUENCE
    fator_ajuste_visitante = 1 - (elo_diff / 1000) * ELO_INFLUENCE

    gols_finais_casa = gols_base_casa * max(0.1, fator_ajuste_casa)
    gols_finais_visitante = gols_base_visitante * max(0.1, fator_ajuste_visitante)

    prob_vitoria_casa, prob_empate, prob_vitoria_visitante = 0, 0, 0
    for g_c in range(POISSON_MAX_GOLS + 1):
        for g_v in range(POISSON_MAX_GOLS + 1):
            prob_placar = poisson(gols_finais_casa, g_c) * poisson(gols_finais_visitante, g_v)
            if g_c > g_v: prob_vitoria_casa += prob_placar
            elif g_c == g_v: prob_empate += prob_placar
            else: prob_vitoria_visitante += prob_placar

    total_prob = prob_vitoria_casa + prob_empate + prob_vitoria_visitante
    if total_prob == 0: return 0.33, 0.33, 0.34

    # Retorna as probabilidades calculadas
    return prob_vitoria_casa / total_prob, prob_empate / total_prob, prob_vitoria_visitante / total_prob

# --- NOVA FUNÇÃO PARA ANALISAR E IMPRIMIR OS ERROS ---

def analisar_e_imprimir_erros(lista_de_erros):
    print("\n--- 🕵️ ANÁLISE DETALHADA DOS ERROS 🕵️ ---")

    zebras_classicas = []
    empates_inesperados = []
    favoritos_desperdicados = []

    for erro in lista_de_erros:
        if erro['resultado'] == 'EMPATE':
            empates_inesperados.append(erro)
        elif erro['previsao'] == 'EMPATE':
            favoritos_desperdicados.append(erro)
        else: # O modelo previu um vencedor, mas o outro venceu
            zebras_classicas.append(erro)

    print(f"\nTotal de erros: {len(lista_de_erros)}")
    print(f"  - Zebras Clássicas: {len(zebras_classicas)} (Modelo previu um vencedor e o outro ganhou)")
    print(f"  - Empates Inesperados: {len(empates_inesperados)} (Modelo previu um vencedor e deu empate)")
    print(f"  - Favoritos Desperdiçados: {len(favoritos_desperdicados)} (Modelo previu empate e alguém ganhou)")

    # Ordena as zebras pela confiança do modelo no time que perdeu, para achar os maiores erros
    zebras_classicas.sort(key=lambda x: x['prob_previsao'], reverse=True)

    print("\n--- EXEMPLOS DOS 5 ERROS MAIS 'SURPREENDENTES' (ZEBRAS) ---")
    print("(Onde o modelo estava mais confiante na sua previsão errada)")
    for i, erro in enumerate(zebras_classicas[:5]):
        print(f"{i+1}. {erro['data'].strftime('%d/%m/%Y')}: {erro['partida']}")
        print(f"   - Placar Real: {erro['placar_real']}")
        print(f"   - Previsão do Modelo: {erro['previsao_txt']} (com {erro['prob_previsao']:.1%} de chance)")
        print(f"   - Resultado Real: {erro['resultado_txt']}")

# --- EXECUÇÃO PRINCIPAL (ORQUESTRADOR DO BACKTESTING) ---
def main():
    try:
        df = pd.read_csv('/kaggle/input/br-data-soccer/campeonato-brasileiro-full.csv')
    except FileNotFoundError:
        print("ERRO: Arquivo 'campeonato-brasileiro-full.csv' não encontrado.")
        return

    df.dropna(subset=['mandante_Placar', 'visitante_Placar', 'data'], inplace=True)
    df['mandante_Placar'] = df['mandante_Placar'].astype(int)
    df['visitante_Placar'] = df['visitante_Placar'].astype(int)
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

    df_treino = df[df['data'].dt.year < ANO_DE_TESTE]
    df_teste = df[df['data'].dt.year == ANO_DE_TESTE].sort_values(by='data')

    if df_teste.empty:
        print(f"ERRO: Nenhum dado encontrado para o ano de teste ({ANO_DE_TESTE}).")
        return

    print(f"Dados divididos: {len(df_treino)} jogos para treino, {len(df_teste)} jogos para teste.")

    print("\nIniciando treinamento com dados até", ANO_DE_TESTE - 1)
    forcas_poisson, medias_liga = calcular_forcas_poisson(df_treino)
    vantagens_casa = calcular_vantagens_casa(df_treino)

    elo_ratings = {}
    for _, partida in df_treino.iterrows():
        # ... (lógica de treino do Elo idêntica à anterior)
        time_c, time_v = partida['mandante'], partida['visitante']
        placar_c, placar_v = partida['mandante_Placar'], partida['visitante_Placar']
        rating_c = elo_ratings.get(time_c, ELO_RATING_INICIAL)
        rating_v = elo_ratings.get(time_v, ELO_RATING_INICIAL)
        vantagem_c = vantagens_casa.get(time_c, ELO_VANTAGEM_CASA_PADRAO)
        novo_rating_c, novo_rating_v = atualizar_ratings_elo(rating_c, rating_v, placar_c, placar_v, vantagem_c)
        elo_ratings[time_c], elo_ratings[time_v] = novo_rating_c, novo_rating_v
    print("Treinamento concluído.")

    print(f"\nIniciando simulação de backtesting para a temporada de {ANO_DE_TESTE}...")
    acertos = 0
    jogos_testados = 0
    erros_do_modelo = [] # NOVA LISTA PARA COLETAR OS ERROS

    for _, partida_teste in df_teste.iterrows():
        jogos_testados += 1
        time_c, time_v = partida_teste['mandante'], partida_teste['visitante']
        placar_c_real, placar_v_real = partida_teste['mandante_Placar'], partida_teste['visitante_Placar']

        prob_vc, prob_e, prob_vv = prever_partida_hibrido(time_c, time_v, elo_ratings, forcas_poisson, medias_liga, vantagens_casa)

        previsao_cod, previsao_txt, prob_previsao = '', '', 0.0
        if prob_vc > prob_e and prob_vc > prob_vv:
            previsao_cod, previsao_txt, prob_previsao = 'VITORIA_CASA', f"Vitória {time_c}", prob_vc
        elif prob_vv > prob_vc and prob_vv > prob_e:
            previsao_cod, previsao_txt, prob_previsao = 'VITORIA_VISITANTE', f"Vitória {time_v}", prob_vv
        else:
            previsao_cod, previsao_txt, prob_previsao = 'EMPATE', "Empate", prob_e

        resultado_cod, resultado_txt = '', ''
        if placar_c_real > placar_v_real:
            resultado_cod, resultado_txt = 'VITORIA_CASA', f"Vitória {time_c}"
        elif placar_v_real > placar_c_real:
            resultado_cod, resultado_txt = 'VITORIA_VISITANTE', f"Vitória {time_v}"
        else:
            resultado_cod, resultado_txt = 'EMPATE', "Empate"

        if previsao_cod == resultado_cod:
            acertos += 1
        else:
            # NOVO: Se errou, adiciona os detalhes do erro à lista
            erro = {
                'data': partida_teste['data'],
                'partida': f"{time_c} vs {time_v}",
                'placar_real': f"{placar_c_real}x{placar_v_real}",
                'previsao_txt': previsao_txt,
                'resultado_txt': resultado_txt,
                'prob_previsao': prob_previsao,
                'previsao': previsao_cod, # para categorização
                'resultado': resultado_cod # para categorização
            }
            erros_do_modelo.append(erro)

        rating_c = elo_ratings.get(time_c, ELO_RATING_INICIAL)
        rating_v = elo_ratings.get(time_v, ELO_RATING_INICIAL)
        vantagem_c = vantagens_casa.get(time_c, ELO_VANTAGEM_CASA_PADRAO)
        novo_rating_c, novo_rating_v = atualizar_ratings_elo(rating_c, rating_v, placar_c_real, placar_v_real, vantagem_c)
        elo_ratings[time_c], elo_ratings[time_v] = novo_rating_c, novo_rating_v

    acuracia = (acertos / jogos_testados) * 100
    print("\n--- ✅ RESULTADO DO BACKTESTING ✅ ---")
    print(f"Acurácia Final: {acuracia:.2f}% ({acertos} acertos de {jogos_testados} jogos)")

    # NOVO: Chama a função para analisar e imprimir os erros coletados
    analisar_e_imprimir_erros(erros_do_modelo)


if __name__ == "__main__":
    main()

Dados divididos: 8025 jogos para treino, 380 jogos para teste.

Iniciando treinamento com dados até 2022


/tmp/ipykernel_35/530192381.py:19: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/530192381.py:19: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/530192381.py:19: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
/tmp/ipykernel_35/530192381.py:19: RuntimeWarning: invalid value encountered in scalar power
  vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)


Treinamento concluído.

Iniciando simulação de backtesting para a temporada de 2023...

--- ✅ RESULTADO DO BACKTESTING ✅ ---
Acurácia Final: 49.21% (187 acertos de 380 jogos)

--- 🕵️ ANÁLISE DETALHADA DOS ERROS 🕵️ ---

Total de erros: 193
  - Zebras Clássicas: 95 (Modelo previu um vencedor e o outro ganhou)
  - Empates Inesperados: 98 (Modelo previu um vencedor e deu empate)
  - Favoritos Desperdiçados: 0 (Modelo previu empate e alguém ganhou)

--- EXEMPLOS DOS 5 ERROS MAIS 'SURPREENDENTES' (ZEBRAS) ---
(Onde o modelo estava mais confiante na sua previsão errada)
1. 29/10/2023: Internacional vs Coritiba
   - Placar Real: 3x4
   - Previsão do Modelo: Vitória Internacional (com 65.6% de chance)
   - Resultado Real: Vitória Coritiba
2. 08/10/2023: Atletico-MG vs Coritiba
   - Placar Real: 1x2
   - Previsão do Modelo: Vitória Atletico-MG (com 64.7% de chance)
   - Resultado Real: Vitória Coritiba
3. 10/05/2023: Internacional vs Athletico-PR
   - Placar Real: 0x2
   - Previsão do Modelo: Vi

# Previsão da rodada 11 de junho até 13 de junho

In [10]:
# -*- coding: utf-8 -*-
import pandas as pd
import math
from datetime import datetime

# --- PARÂMETROS OTIMIZADOS DO NOSSO MODELO CAMPEÃO ---
ELO_RATING_INICIAL = 1500
ELO_K_FACTOR_BASE = 30
ELO_VANTAGEM_CASA_PADRAO = 80
POISSON_MAX_GOLS = 8
ELO_INFLUENCE = 0.35

# --- FUNÇÕES DO MODELO (As mesmas que validamos anteriormente) ---

def calcular_vantagens_casa(df):
    vantagens = {}
    times = pd.unique(df['mandante'])
    for time in times:
        jogos_casa = df[df['mandante'] == time]
        if not jogos_casa.empty:
            saldo_de_gols_medio = (jogos_casa['mandante_Placar'] - jogos_casa['visitante_Placar']).mean()
            
            # NOVO: Verifica se o saldo é positivo ANTES de calcular a potência.
            if saldo_de_gols_medio > 0:
                # Se o saldo é positivo, calcula a vantagem normalmente.
                vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
            else:
                # Se o time tem saldo negativo ou zero em casa, sua vantagem de casa é considerada zero.
                vantagem_rating = 0
            
            vantagens[time] = vantagem_rating
            
    return vantagens

def atualizar_ratings_elo(rating_casa, rating_visitante, placar_casa, placar_visitante, vantagem_casa_time):
    if placar_casa > placar_visitante:
        resultado_real_casa, margem_gols = 1.0, placar_casa - placar_visitante
    elif placar_casa < placar_visitante:
        resultado_real_casa, margem_gols = 0.0, placar_visitante - placar_casa
    else:
        resultado_real_casa, margem_gols = 0.5, 0

    k_ajustado = ELO_K_FACTOR_BASE * math.log(margem_gols + 1)
    if resultado_real_casa == 0.5: k_ajustado = ELO_K_FACTOR_BASE
    expectativa_casa = 1 / (1 + 10 ** ((rating_visitante - (rating_casa + vantagem_casa_time)) / 400))
    novo_rating_casa = rating_casa + k_ajustado * (resultado_real_casa - expectativa_casa)
    novo_rating_visitante = rating_visitante + k_ajustado * ((1.0 - resultado_real_casa) - (1 - expectativa_casa))
    return novo_rating_casa, novo_rating_visitante

def calcular_forcas_poisson(df):
    forcas = {}
    media_gols_marcados_casa = df['mandante_Placar'].mean()
    media_gols_sofridos_casa = df['visitante_Placar'].mean()
    media_gols_marcados_fora = df['visitante_Placar'].mean()
    media_gols_sofridos_fora = df['mandante_Placar'].mean()
    times = pd.unique(df[['mandante', 'visitante']].values.ravel('K'))

    for time in times:
        forcas_time = {
            'ataque_casa': (df[df['mandante'] == time]['mandante_Placar'].mean() / media_gols_marcados_casa),
            'defesa_casa': (df[df['mandante'] == time]['visitante_Placar'].mean() / media_gols_sofridos_casa),
            'ataque_fora': (df[df['visitante'] == time]['visitante_Placar'].mean() / media_gols_marcados_fora),
            'defesa_fora': (df[df['visitante'] == time]['mandante_Placar'].mean() / media_gols_sofridos_fora)
        }
        # Substitui NaN (caso um time não tenha jogado em casa/fora) pelo valor neutro 1.0
        forcas[time] = {k: v if not pd.isna(v) else 1.0 for k, v in forcas_time.items()}

    medias_liga = {'gols_casa': media_gols_marcados_casa, 'gols_fora': media_gols_marcados_fora}
    return forcas, medias_liga

def poisson(gols_esperados, num_gols):
    if gols_esperados < 0: return 0
    if num_gols > 30: return 0
    return (gols_esperados ** num_gols) * math.exp(-gols_esperados) / math.factorial(num_gols)

def prever_partida_hibrido(time_casa, time_visitante, elo_ratings, forcas_poisson, medias_liga, vantagens_casa):
    if time_casa not in forcas_poisson or time_visitante not in forcas_poisson:
        return 0.33, 0.33, 0.34

    ataque_casa = forcas_poisson[time_casa]['ataque_casa']
    defesa_visitante = forcas_poisson[time_visitante]['defesa_fora']
    gols_base_casa = ataque_casa * defesa_visitante * medias_liga['gols_casa']
    ataque_visitante = forcas_poisson[time_visitante]['ataque_fora']
    defesa_casa = forcas_poisson[time_casa]['defesa_casa']
    gols_base_visitante = ataque_visitante * defesa_casa * medias_liga['gols_fora']
    rating_casa = elo_ratings.get(time_casa, ELO_RATING_INICIAL)
    rating_visitante = elo_ratings.get(time_visitante, ELO_RATING_INICIAL)
    vantagem_time_casa = vantagens_casa.get(time_casa, ELO_VANTAGEM_CASA_PADRAO)
    elo_diff = (rating_casa + vantagem_time_casa) - rating_visitante
    fator_ajuste_casa = 1 + (elo_diff / 1000) * ELO_INFLUENCE
    fator_ajuste_visitante = 1 - (elo_diff / 1000) * ELO_INFLUENCE
    gols_finais_casa = gols_base_casa * max(0.1, fator_ajuste_casa)
    gols_finais_visitante = gols_base_visitante * max(0.1, fator_ajuste_visitante)
    prob_vitoria_casa, prob_empate, prob_vitoria_visitante = 0, 0, 0
    for g_c in range(POISSON_MAX_GOLS + 1):
        for g_v in range(POISSON_MAX_GOLS + 1):
            prob_placar = poisson(gols_finais_casa, g_c) * poisson(gols_finais_visitante, g_v)
            if g_c > g_v: prob_vitoria_casa += prob_placar
            elif g_c == g_v: prob_empate += prob_placar
            else: prob_vitoria_visitante += prob_placar
    total_prob = prob_vitoria_casa + prob_empate + prob_vitoria_visitante
    if total_prob == 0: return 0.33, 0.33, 0.34
    return prob_vitoria_casa / total_prob, prob_empate / total_prob, prob_vitoria_visitante / total_prob

# --- FUNÇÃO DE APRESENTAÇÃO ---

def gerar_previsao_formatada(time_casa, time_visitante, elo_ratings, forcas_poisson, medias_liga, vantagens_casa):
    """Chama o modelo e imprime a previsão de forma organizada."""
    prob_vc, prob_e, prob_vv = prever_partida_hibrido(time_casa, time_visitante, elo_ratings, forcas_poisson, medias_liga, vantagens_casa)

    rating_casa = elo_ratings.get(time_casa, ELO_RATING_INICIAL)
    rating_visitante = elo_ratings.get(time_visitante, ELO_RATING_INICIAL)

    print("\n------------------------------------------------------")
    print(f"🔮 PREVISÃO PARA: {time_casa.upper()} vs {time_visitante.upper()}")
    print("------------------------------------------------------")
    print(f"Rating Elo (Fase Atual): {int(rating_casa)} vs {int(rating_visitante)}")
    print(f"Chance de Vitória {time_casa}: {prob_vc:.1%}")
    print(f"Chance de Empate: {prob_e:.1%}")
    print(f"Chance de Vitória {time_visitante}: {prob_vv:.1%}")
    print("------------------------------------------------------")

# --- EXECUÇÃO PRINCIPAL ---
def main():
    ANO_ATUAL = datetime.now().year

    try:
        df = pd.read_csv('/kaggle/input/br-data-soccer/campeonato-brasileiro-full.csv')
    except FileNotFoundError:
        print("ERRO: Arquivo 'campeonato-brasileiro-full.csv' não encontrado.")
        return

    df.dropna(subset=['mandante_Placar', 'visitante_Placar', 'data'], inplace=True)
    df['mandante_Placar'] = df['mandante_Placar'].astype(int)
    df['visitante_Placar'] = df['visitante_Placar'].astype(int)
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

    # 1. TREINAMENTO HISTÓRICO (Tudo antes do ano atual)
    print(f"Iniciando treinamento com dados históricos até {ANO_ATUAL - 1}...")
    df_historico = df[df['data'].dt.year < ANO_ATUAL]
    forcas_poisson, medias_liga = calcular_forcas_poisson(df_historico)
    vantagens_casa = calcular_vantagens_casa(df_historico)

    elo_ratings = {}
    for _, partida in df_historico.iterrows():
        time_c, time_v = partida['mandante'], partida['visitante']
        placar_c, placar_v = partida['mandante_Placar'], partida['visitante_Placar']
        rating_c = elo_ratings.get(time_c, ELO_RATING_INICIAL)
        rating_v = elo_ratings.get(time_v, ELO_RATING_INICIAL)
        vantagem_c = vantagens_casa.get(time_c, ELO_VANTAGEM_CASA_PADRAO)
        novo_rating_c, novo_rating_v = atualizar_ratings_elo(rating_c, rating_v, placar_c, placar_v, vantagem_c)
        elo_ratings[time_c], elo_ratings[time_v] = novo_rating_c, novo_rating_v
    print("Treinamento histórico concluído.")

    # 2. ATUALIZAÇÃO COM TEMPORADA ATUAL (Jogos que já aconteceram no ano atual)
    print(f"\nAtualizando o modelo com os jogos já realizados em {ANO_ATUAL}...")
    df_atualizacao = df[df['data'].dt.year == ANO_ATUAL].sort_values(by='data')
    for _, partida in df_atualizacao.iterrows():
        time_c, time_v = partida['mandante'], partida['visitante']
        placar_c, placar_v = partida['mandante_Placar'], partida['visitante_Placar']
        rating_c = elo_ratings.get(time_c, ELO_RATING_INICIAL)
        rating_v = elo_ratings.get(time_v, ELO_RATING_INICIAL)
        vantagem_c = vantagens_casa.get(time_c, ELO_VANTAGEM_CASA_PADRAO)
        novo_rating_c, novo_rating_v = atualizar_ratings_elo(rating_c, rating_v, placar_c, placar_v, vantagem_c)
        elo_ratings[time_c], elo_ratings[time_v] = novo_rating_c, novo_rating_v
    print("Ratings Elo atualizados para a data de hoje!")

    # 3. PREVISÃO DA PRÓXIMA RODADA
    # Altere a lista de jogos abaixo para prever qualquer partida que desejar!
    jogos_para_prever = [
        ("Bragantino", "Bahia"),
        ("Vitoria", "Cruzeiro"),
        ("Fortaleza", "Santos"),
        ("Gremio", "Corinthians"),
        ("Sao Paulo", "Vasco"),
        ("Atletico MG", "Internacional")
    ]

    for mandante, visitante in jogos_para_prever:
        gerar_previsao_formatada(mandante, visitante, elo_ratings, forcas_poisson, medias_liga, vantagens_casa)

if __name__ == "__main__":
    main()

Iniciando treinamento com dados históricos até 2024...
Treinamento histórico concluído.

Atualizando o modelo com os jogos já realizados em 2025...
Ratings Elo atualizados para a data de hoje!

------------------------------------------------------
🔮 PREVISÃO PARA: BRAGANTINO vs BAHIA
------------------------------------------------------
Rating Elo (Fase Atual): 1553 vs 1550
Chance de Vitória Bragantino: 53.7%
Chance de Empate: 25.0%
Chance de Vitória Bahia: 21.3%
------------------------------------------------------

------------------------------------------------------
🔮 PREVISÃO PARA: VITORIA vs CRUZEIRO
------------------------------------------------------
Rating Elo (Fase Atual): 1555 vs 1547
Chance de Vitória Vitoria: 37.6%
Chance de Empate: 25.6%
Chance de Vitória Cruzeiro: 36.7%
------------------------------------------------------

------------------------------------------------------
🔮 PREVISÃO PARA: FORTALEZA vs SANTOS
--------------------------------------------------

- Rastrear estatísticas por time: criar um "placar" para cada time. Durante a simulação de 2023, para cada jogo, vamos registrar se o modelo acertou ou errou a previsão para os dois times envolvidos.
- Calcular a performance: no final do backtest, teremos o número total de acertos e erros para cada time do campeonato.
Apresentar os Rankings: Vamos criar dois rankings:

- Top 5 - times mais "previsíveis": os 5 times com a maior porcentagem de acerto do modelo.
- Top 5 - times mais "imprevisíveis": os 5 times com o maior número absoluto de erros do modelo.

In [11]:
# -*- coding: utf-8 -*-
import pandas as pd
import math
from datetime import datetime

# --- PARÂMETROS OTIMIZADOS ---
ELO_RATING_INICIAL = 1500
ELO_K_FACTOR_BASE = 30
ELO_VANTAGEM_CASA_PADRAO = 80
POISSON_MAX_GOLS = 8
ELO_INFLUENCE = 0.35
ANO_DE_TESTE = 2023

# --- FUNÇÕES DO MODELO (As mesmas, com a correção do RuntimeWarning) ---

def calcular_vantagens_casa(df):
    vantagens = {}
    times = pd.unique(df['mandante'])
    for time in times:
        jogos_casa = df[df['mandante'] == time]
        if not jogos_casa.empty:
            saldo_de_gols_medio = (jogos_casa['mandante_Placar'] - jogos_casa['visitante_Placar']).mean()
            if saldo_de_gols_medio > 0:
                vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
            else:
                vantagem_rating = 0
            vantagens[time] = vantagem_rating
    return vantagens

# ... (As outras funções de cálculo: atualizar_ratings_elo, calcular_forcas_poisson, poisson, 
#      prever_partida_hibrido, continuam exatamente as mesmas do script anterior)
def atualizar_ratings_elo(rating_casa, rating_visitante, placar_casa, placar_visitante, vantagem_casa_time):
    if placar_casa > placar_visitante:
        resultado_real_casa, margem_gols = 1.0, placar_casa - placar_visitante
    elif placar_casa < placar_visitante:
        resultado_real_casa, margem_gols = 0.0, placar_visitante - placar_casa
    else:
        resultado_real_casa, margem_gols = 0.5, 0

    k_ajustado = ELO_K_FACTOR_BASE * math.log(margem_gols + 1)
    if resultado_real_casa == 0.5: k_ajustado = ELO_K_FACTOR_BASE
    expectativa_casa = 1 / (1 + 10 ** ((rating_visitante - (rating_casa + vantagem_casa_time)) / 400))
    novo_rating_casa = rating_casa + k_ajustado * (resultado_real_casa - expectativa_casa)
    novo_rating_visitante = rating_visitante + k_ajustado * ((1.0 - resultado_real_casa) - (1 - expectativa_casa))
    return novo_rating_casa, novo_rating_visitante

def calcular_forcas_poisson(df):
    forcas = {}
    media_gols_marcados_casa = df['mandante_Placar'].mean()
    media_gols_sofridos_casa = df['visitante_Placar'].mean()
    media_gols_marcados_fora = df['visitante_Placar'].mean()
    media_gols_sofridos_fora = df['mandante_Placar'].mean()
    times = pd.unique(df[['mandante', 'visitante']].values.ravel('K'))
    for time in times:
        forcas_time = {
            'ataque_casa': (df[df['mandante'] == time]['mandante_Placar'].mean() / media_gols_marcados_casa),
            'defesa_casa': (df[df['mandante'] == time]['visitante_Placar'].mean() / media_gols_sofridos_casa),
            'ataque_fora': (df[df['visitante'] == time]['visitante_Placar'].mean() / media_gols_marcados_fora),
            'defesa_fora': (df[df['visitante'] == time]['mandante_Placar'].mean() / media_gols_sofridos_fora)
        }
        forcas[time] = {k: v if not pd.isna(v) else 1.0 for k, v in forcas_time.items()}
    medias_liga = {'gols_casa': media_gols_marcados_casa, 'gols_fora': media_gols_marcados_fora}
    return forcas, medias_liga

def poisson(gols_esperados, num_gols):
    if gols_esperados < 0: return 0
    if num_gols > 30: return 0 
    return (gols_esperados ** num_gols) * math.exp(-gols_esperados) / math.factorial(num_gols)

def prever_partida_hibrido(time_casa, time_visitante, elo_ratings, forcas_poisson, medias_liga, vantagens_casa):
    if time_casa not in forcas_poisson or time_visitante not in forcas_poisson:
        return 0.33, 0.33, 0.34
    
    # Adicionando uma verificação mais robusta para NaN
    if any(pd.isna(forcas_poisson[time_casa].get(key)) for key in ['ataque_casa', 'defesa_casa']) or \
       any(pd.isna(forcas_poisson[time_visitante].get(key)) for key in ['ataque_fora', 'defesa_fora']):
        return 0.33, 0.33, 0.34

    ataque_casa = forcas_poisson[time_casa]['ataque_casa']
    defesa_visitante = forcas_poisson[time_visitante]['defesa_fora']
    gols_base_casa = ataque_casa * defesa_visitante * medias_liga['gols_casa']
    ataque_visitante = forcas_poisson[time_visitante]['ataque_fora']
    defesa_casa = forcas_poisson[time_casa]['defesa_casa']
    gols_base_visitante = ataque_visitante * defesa_casa * medias_liga['gols_fora']
    rating_casa = elo_ratings.get(time_casa, ELO_RATING_INICIAL)
    rating_visitante = elo_ratings.get(time_visitante, ELO_RATING_INICIAL)
    vantagem_time_casa = vantagens_casa.get(time_casa, ELO_VANTAGEM_CASA_PADRAO)
    elo_diff = (rating_casa + vantagem_time_casa) - rating_visitante
    fator_ajuste_casa = 1 + (elo_diff / 1000) * ELO_INFLUENCE
    fator_ajuste_visitante = 1 - (elo_diff / 1000) * ELO_INFLUENCE
    gols_finais_casa = gols_base_casa * max(0.1, fator_ajuste_casa)
    gols_finais_visitante = gols_base_visitante * max(0.1, fator_ajuste_visitante)
    prob_vitoria_casa, prob_empate, prob_vitoria_visitante = 0, 0, 0
    for g_c in range(POISSON_MAX_GOLS + 1):
        for g_v in range(POISSON_MAX_GOLS + 1):
            prob_placar = poisson(gols_finais_casa, g_c) * poisson(gols_finais_visitante, g_v)
            if g_c > g_v: prob_vitoria_casa += prob_placar
            elif g_c == g_v: prob_empate += prob_placar
            else: prob_vitoria_visitante += prob_placar
    total_prob = prob_vitoria_casa + prob_empate + prob_vitoria_visitante
    if total_prob == 0: return 0.33, 0.33, 0.34
    return prob_vitoria_casa / total_prob, prob_empate / total_prob, prob_vitoria_visitante / total_prob

# --- NOVA FUNÇÃO PARA ANALISAR PERFORMANCE POR TIME ---

def analisar_performance_por_time(stats_times):
    print("\n--- 📊 ANÁLISE DE PERFORMANCE POR TIME 📊 ---")
    
    # Converte o dicionário para um DataFrame do pandas para facilitar a manipulação
    df_stats = pd.DataFrame.from_dict(stats_times, orient='index')
    
    # Calcula a porcentagem de acerto
    df_stats['percentual_acerto'] = (df_stats['acertos'] / df_stats['total_jogos']) * 100
    
    # Ordena para encontrar os melhores e piores
    df_mais_previsiveis = df_stats.sort_values(by='percentual_acerto', ascending=False)
    df_mais_imprevisiveis = df_stats.sort_values(by='erros', ascending=False)
    
    print("\n✅ TOP 5 TIMES MAIS 'PREVISÍVEIS' (Maior % de Acerto do Modelo)")
    print("-----------------------------------------------------------------")
    for index, row in df_mais_previsiveis.head(5).iterrows():
        print(f"- {index}: {row['percentual_acerto']:.2f}% de acerto ({row['acertos']} de {row['total_jogos']} jogos)")

    print("\n❌ TOP 5 TIMES MAIS 'IMPREVISÍVEIS' (Maior Nº de Erros do Modelo)")
    print("----------------------------------------------------------------")
    for index, row in df_mais_imprevisiveis.head(5).iterrows():
        print(f"- {index}: {row['erros']} erros em {row['total_jogos']} jogos ({row['percentual_acerto']:.2f}% de acerto)")
        
# --- EXECUÇÃO PRINCIPAL (ORQUESTRADOR DO BACKTESTING) ---
def main():
    try:
        df = pd.read_csv('/kaggle/input/br-data-soccer/campeonato-brasileiro-full.csv')
    except FileNotFoundError:
        print("ERRO: Arquivo 'campeonato-brasileiro-estatisticas-full.csv' não encontrado.")
        return
        
    df.dropna(subset=['mandante_Placar', 'visitante_Placar', 'data'], inplace=True)
    df['mandante_Placar'] = df['mandante_Placar'].astype(int)
    df['visitante_Placar'] = df['visitante_Placar'].astype(int)
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

    df_treino = df[df['data'].dt.year < ANO_DE_TESTE]
    df_teste = df[df['data'].dt.year == ANO_DE_TESTE].sort_values(by='data')

    if df_teste.empty:
        print(f"ERRO: Nenhum dado encontrado para o ano de teste ({ANO_DE_TESTE}).")
        return

    print(f"Dados divididos: {len(df_treino)} jogos para treino, {len(df_teste)} jogos para teste.")

    print("\nIniciando treinamento com dados até", ANO_DE_TESTE - 1)
    forcas_poisson, medias_liga = calcular_forcas_poisson(df_treino)
    vantagens_casa = calcular_vantagens_casa(df_treino)
    
    elo_ratings = {}
    for _, partida in df_treino.iterrows():
        time_c, time_v = partida['mandante'], partida['visitante']
        placar_c, placar_v = partida['mandante_Placar'], partida['visitante_Placar']
        rating_c = elo_ratings.get(time_c, ELO_RATING_INICIAL)
        rating_v = elo_ratings.get(time_v, ELO_RATING_INICIAL)
        vantagem_c = vantagens_casa.get(time_c, ELO_VANTAGEM_CASA_PADRAO)
        novo_rating_c, novo_rating_v = atualizar_ratings_elo(rating_c, rating_v, placar_c, placar_v, vantagem_c)
        elo_ratings[time_c], elo_ratings[time_v] = novo_rating_c, novo_rating_v
    print("Treinamento concluído.")

    print(f"\nIniciando simulação de backtesting para a temporada de {ANO_DE_TESTE}...")
    acertos_gerais = 0
    # NOVO: Dicionário para guardar as estatísticas de cada time
    estatisticas_times = {}

    for _, partida_teste in df_teste.iterrows():
        time_c, time_v = partida_teste['mandante'], partida_teste['visitante']
        placar_c_real, placar_v_real = partida_teste['mandante_Placar'], partida_teste['visitante_Placar']

        prob_vc, prob_e, prob_vv = prever_partida_hibrido(time_c, time_v, elo_ratings, forcas_poisson, medias_liga, vantagens_casa)
        
        previsao_cod = ''
        if prob_vc > prob_e and prob_vc > prob_vv: previsao_cod = 'VITORIA_CASA'
        elif prob_vv > prob_vc and prob_vv > prob_e: previsao_cod = 'VITORIA_VISITANTE'
        else: previsao_cod = 'EMPATE'

        resultado_cod = ''
        if placar_c_real > placar_v_real: resultado_cod = 'VITORIA_CASA'
        elif placar_v_real > placar_c_real: resultado_cod = 'VITORIA_VISITANTE'
        else: resultado_cod = 'EMPATE'

        # NOVO: Atualiza as estatísticas para os dois times envolvidos
        for time in [time_c, time_v]:
            if time not in estatisticas_times:
                estatisticas_times[time] = {'acertos': 0, 'erros': 0, 'total_jogos': 0}
        
        estatisticas_times[time_c]['total_jogos'] += 1
        estatisticas_times[time_v]['total_jogos'] += 1

        if previsao_cod == resultado_cod:
            acertos_gerais += 1
            estatisticas_times[time_c]['acertos'] += 1
            estatisticas_times[time_v]['acertos'] += 1
        else:
            estatisticas_times[time_c]['erros'] += 1
            estatisticas_times[time_v]['erros'] += 1

        rating_c = elo_ratings.get(time_c, ELO_RATING_INICIAL)
        rating_v = elo_ratings.get(time_v, ELO_RATING_INICIAL)
        vantagem_c = vantagens_casa.get(time_c, ELO_VANTAGEM_CASA_PADRAO)
        novo_rating_c, novo_rating_v = atualizar_ratings_elo(rating_c, rating_v, placar_c_real, placar_v_real, vantagem_c)
        elo_ratings[time_c], elo_ratings[time_v] = novo_rating_c, novo_rating_v
    
    acuracia = (acertos_gerais / len(df_teste)) * 100
    print("\n--- ✅ RESULTADO GERAL DO BACKTESTING ✅ ---")
    print(f"Acurácia Final: {acuracia:.2f}% ({acertos_gerais} acertos de {len(df_teste)} jogos)")
    
    # NOVO: Chama a função para analisar e imprimir a performance por time
    analisar_performance_por_time(estatisticas_times)


if __name__ == "__main__":
    main()

Dados divididos: 8025 jogos para treino, 380 jogos para teste.

Iniciando treinamento com dados até 2022
Treinamento concluído.

Iniciando simulação de backtesting para a temporada de 2023...

--- ✅ RESULTADO GERAL DO BACKTESTING ✅ ---
Acurácia Final: 49.21% (187 acertos de 380 jogos)

--- 📊 ANÁLISE DE PERFORMANCE POR TIME 📊 ---

✅ TOP 5 TIMES MAIS 'PREVISÍVEIS' (Maior % de Acerto do Modelo)
-----------------------------------------------------------------
- Gremio: 65.79% de acerto (25.0 de 38.0 jogos)
- Fluminense: 63.16% de acerto (24.0 de 38.0 jogos)
- Palmeiras: 60.53% de acerto (23.0 de 38.0 jogos)
- America-MG: 57.89% de acerto (22.0 de 38.0 jogos)
- Sao Paulo: 55.26% de acerto (21.0 de 38.0 jogos)

❌ TOP 5 TIMES MAIS 'IMPREVISÍVEIS' (Maior Nº de Erros do Modelo)
----------------------------------------------------------------
- Cruzeiro: 26.0 erros em 38.0 jogos (31.58% de acerto)
- Atletico-MG: 23.0 erros em 38.0 jogos (39.47% de acerto)
- Santos: 22.0 erros em 38.0 jogos (42.

Adicionando os dados do campeonato brasileiro de 2025 (12 de junho)

## Adicionados os dados das rodadas do campeonato de 2025 ao arquivo original (2003-2024)

In [12]:
import pandas as pd

# Carregando o dataset fornecido
file_path = '/kaggle/input/br-2025/campeonato-brasileiro-full.csv'
df = pd.read_csv(file_path)

# Verificando nomes das colunas e primeiras linhas para inspeção
colunas = df.columns.tolist()
primeiras_linhas = df.head()

# Normalizando nomes das equipes
df['mandante'] = df['mandante'].str.strip().str.lower()
df['visitante'] = df['visitante'].str.strip().str.lower()

# Exibindo resultado para verificação
colunas, primeiras_linhas.head()


(['mandante', 'visitante', 'resultado', 'gols_mandante', 'gols_visitante'],
         mandante    visitante    resultado  gols_mandante  gols_visitante
 0        guarani        vasco      guarani              4               2
 1    atletico pr       gremio  atletico pr              2               0
 2       flamengo     coritiba       empate              1               1
 3          goias     paysandu       empate              2               2
 4  internacional  ponte preta       empate              1               1)

### Recalculo do ELO

In [13]:
import math
from datetime import timedelta

# Parâmetros
ELO_INICIAL = 1500
K_BASE = 20
VANTAGEM_CASA = 0
MAX_GOLS = 8
INFLUENCE = 0.35

def calcular_vantagens(df):
    v = {}
    for t in pd.unique(df['mandante']):
        saldo = (df[df['mandante']==t]['gols_mandante'] -
                 df[df['mandante']==t]['gols_visitante']).mean()
        v[t] = 60 * (saldo**0.8) if saldo > 0 else 0
    return v

def atualiza_elo(r_c, r_v, g_c, g_v, v_c):
    if g_c > g_v:
        res, diff = 1.0, g_c - g_v
    elif g_c < g_v:
        res, diff = 0.0, g_v - g_c
    else:
        res, diff = 0.5, 0
    k = K_BASE * math.log(diff + 1) if res != 0.5 else K_BASE
    e_c = 1 / (1 + 10 ** ((r_v - (r_c + v_c)) / 400))
    return r_c + k * (res - e_c), r_v + k * ((1 - res) - (1 - e_c))

# Ordena o dataframe por data se existir coluna 'data'
if 'data' in df.columns:
    df['data'] = pd.to_datetime(df['data'], dayfirst=True)
    df = df.sort_values('data')

# Calcula vantagens
vant = calcular_vantagens(df)

# Calcula Elo ratings históricos
elo = {}
for _, r in df.iterrows():
    nc, nv = atualiza_elo(
        elo.get(r['mandante'], ELO_INICIAL),
        elo.get(r['visitante'], ELO_INICIAL),
        r['gols_mandante'], r['gols_visitante'],
        vant.get(r['mandante'], VANTAGEM_CASA)
    )
    elo[r['mandante']], elo[r['visitante']] = nc, nv

# Exibir alguns ratings finais para validação
elo_final = {k: round(v, 2) for k, v in list(elo.items())[:10]}
elo_final


{'guarani': 1437.13,
 'vasco': 1530.13,
 'atletico pr': 1518.46,
 'gremio': 1510.5,
 'flamengo': 1716.98,
 'coritiba': 1428.98,
 'goias': 1475.31,
 'paysandu': 1411.27,
 'internacional': 1636.93,
 'ponte preta': 1482.35}

## Valida o CSV

In [14]:
import pandas as pd

df = pd.read_csv('/kaggle/input/br-2025/campeonato-brasileiro-full.csv')
df['mandante'] = df['mandante'].str.strip().str.lower()
df['visitante'] = df['visitante'].str.strip().str.lower()

clubes_foco = [
    'flamengo', 'fluminense', 'vasco', 'botafogo',
    'palmeiras', 'sao paulo', 'santos', 'corinthians',
    'atletico mineiro', 'cruzeiro', 'gremio', 'internacional'
]

# Nomes únicos
print('Nomes únicos (mandante):', sorted(df['mandante'].unique()))
print('Nomes únicos (visitante):', sorted(df['visitante'].unique()))

# Quantidade de jogos por clube foco
dados = []
for clube in clubes_foco:
    mandante = len(df[df['mandante'] == clube])
    visitante = len(df[df['visitante'] == clube])
    dados.append({'clube': clube, 'mandante': mandante, 'visitante': visitante})
df_qtd = pd.DataFrame(dados)
print('\nJogos por clube foco:\n', df_qtd)

# Amostra dos dados para os clubes foco
amostra = df[df['mandante'].isin(clubes_foco) | df['visitante'].isin(clubes_foco)].head(10)
print('\nAmostra dos dados (clubes foco):\n', amostra)


Nomes únicos (mandante): ['america mineiro', 'america rn', 'atletico goianense', 'atletico mineiro', 'atletico pr', 'avai', 'bahia', 'barueri', 'botafogo', 'bragantino', 'brasiliense', 'ceara', 'chapecoense', 'corinthians', 'coritiba', 'criciuma', 'cruzeiro', 'csa', 'cuiaba', 'figueirense', 'flamengo', 'fluminense', 'fortaleza', 'goias', 'gremio', 'gremio prudente', 'guarani', 'internacional', 'ipatinga', 'joinville', 'juventude', 'mirassol', 'nautico', 'palmeiras', 'parana', 'paysandu', 'ponte preta', 'portuguesa', 'santa cruz', 'santos', 'santos andre', 'sao caetano', 'sao paulo', 'sport', 'vasco', 'vitoria']
Nomes únicos (visitante): ['america mineiro', 'america rn', 'atletico goianense', 'atletico mineiro', 'atletico pr', 'avai', 'bahia', 'barueri', 'botafogo', 'bragantino', 'brasiliense', 'ceara', 'chapecoense', 'corinthians', 'coritiba', 'criciuma', 'cruzeiro', 'csa', 'cuiaba', 'figueirense', 'flamengo', 'fluminense', 'fortaleza', 'goias', 'gremio', 'gremio prudente', 'guarani', 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

# Clubes-foco
clubes_foco = [
    'flamengo', 'fluminense', 'vasco', 'botafogo',
    'palmeiras', 'sao paulo', 'santos', 'corinthians',
    'atletico mineiro', 'cruzeiro', 'gremio', 'internacional'
]

# Carregar e normalizar dados
df = pd.read_csv('/kaggle/input/br-data-soccer/campeonato-brasileiro-full.csv')
df['mandante'] = df['mandante'].str.strip().str.lower()
df['visitante'] = df['visitante'].str.strip().str.lower()
df['vencedor'] = df['vencedor'].str.strip().str.lower()

if 'data' in df.columns:
    df['data'] = pd.to_datetime(df['data'], dayfirst=True)
    df = df.sort_values('data')
    df['ano'] = df['data'].dt.year
else:
    df['ano'] = 2000

# Parâmetros ELO
ELO_INICIAL = 1500
K_BASE = 20
VANTAGEM_CASA = 0

def calcular_vantagens(df):
    v = {}
    for t in pd.unique(df['mandante']):
        saldo = (df[df['mandante']==t]['mandante_Placar'] - df[df['mandante']==t]['visitante_Placar']).mean()
        v[t] = 60 * (saldo**0.8) if saldo > 0 else 0
    return v

def atualiza_elo(r_c, r_v, g_c, g_v, v_c):
    if g_c > g_v:
        res, diff = 1.0, g_c - g_v
    elif g_c < g_v:
        res, diff = 0.0, g_v - g_c
    else:
        res, diff = 0.5, 0
    k = K_BASE * math.log(diff + 1) if res != 0.5 else K_BASE
    e_c = 1 / (1 + 10 ** ((r_v - (r_c + v_c)) / 400))
    return r_c + k * (res - e_c), r_v + k * ((1 - res) - (1 - e_c))

anos_validos = sorted(df['ano'].unique())
elos = {t: {ano: ELO_INICIAL for ano in anos_validos} for t in clubes_foco}
elo_atual = {t: ELO_INICIAL for t in clubes_foco}
vant = calcular_vantagens(df)
for _, row in df.iterrows():
    ano = row['ano']
    m, v = row['mandante'], row['visitante']
    if m in clubes_foco or v in clubes_foco:
        em = elo_atual.get(m, ELO_INICIAL)
        ev = elo_atual.get(v, ELO_INICIAL)
        nc, nv = atualiza_elo(em, ev, row['mandante_Placar'], row['visitante_Placar'], vant.get(m, VANTAGEM_CASA))
        elo_atual[m], elo_atual[v] = nc, nv
        if m in clubes_foco:
            elos[m][ano] = nc
        if v in clubes_foco:
            elos[v][ano] = nv
df_elos = pd.DataFrame(elos).T[anos_validos].T  # anos como índice, clubes como colunas

# Insights
crescimento = df_elos.iloc[-1] - df_elos.iloc[0]
maior_crescimento = crescimento.idxmax()
maior_crescimento_valor = crescimento.max()

# Percentual de vitórias mandante
df_vit_mandante = df[df['mandante'].isin(clubes_foco)]
percent_vit = (df_vit_mandante['vencedor'] == df_vit_mandante['mandante']).groupby(df_vit_mandante['mandante']).mean()

# Percentual de empates como visitante
empates_fora = df[(df['visitante'].isin(clubes_foco)) & (df['vencedor'] == '-')].groupby('visitante').size()
total_fora = df[df['visitante'].isin(clubes_foco)].groupby('visitante').size()
percent_empate_fora = (empates_fora / total_fora).fillna(0)

# Confrontos grandes mais desequilibrados
confrontos = df[df['mandante'].isin(clubes_foco) & df['visitante'].isin(clubes_foco)]
grupo = confrontos.groupby(['mandante','visitante']).agg({
    'mandante_Placar':'sum','visitante_Placar':'sum','vencedor':'count'
})
grupo['diff_media_gols'] = (grupo['mandante_Placar'] - grupo['visitante_Placar']).abs() / grupo['vencedor']
top_confrontos = grupo['diff_media_gols'].sort_values(ascending=False).head(5)

# Markdown de insights
print("""
**Insights sobre os grandes clubes do Brasileirão**

- Clube com maior crescimento de Elo no período analisado: **{}** (+{:.2f} pontos)
- Percentual de vitórias como mandante:
{}
- Percentual de empates como visitante:
{}
- Confrontos grandes com maior desequilíbrio (diferença média de gols):
{}
""".format(
    maior_crescimento.title(),
    maior_crescimento_valor,
    percent_vit.sort_values(ascending=False).to_markdown(),
    percent_empate_fora.sort_values(ascending=False).to_markdown(),
    top_confrontos.to_markdown()
))

# Gráfico de evolução do Elo
plt.figure(figsize=(12,7))
for clube in clubes_foco:
    plt.plot(df_elos.index, df_elos[clube], label=clube.title())
plt.title('Evolução do Elo dos Grandes Clubes (histórico completo)')
plt.xlabel('Ano')
plt.ylabel('Rating Elo')
plt.legend()
plt.grid(True)
plt.show()



**Insights sobre os grandes clubes do Brasileirão**

- Clube com maior crescimento de Elo no período analisado: **Palmeiras** (+172.08 pontos)
- Percentual de vitórias como mandante:
| mandante      |        0 |
|:--------------|---------:|
| internacional | 0.586797 |
| gremio        | 0.585052 |
| palmeiras     | 0.580311 |
| sao paulo     | 0.574766 |
| santos        | 0.562347 |
| flamengo      | 0.550351 |
| corinthians   | 0.540342 |
| cruzeiro      | 0.533693 |
| fluminense    | 0.495327 |
| vasco         | 0.447447 |
- Percentual de empates como visitante:
| visitante     |        0 |
|:--------------|---------:|
| corinthians   | 0.300733 |
| palmeiras     | 0.300518 |
| vasco         | 0.294294 |
| sao paulo     | 0.287383 |
| flamengo      | 0.284382 |
| internacional | 0.281174 |
| gremio        | 0.275773 |
| santos        | 0.271394 |
| fluminense    | 0.242991 |
| cruzeiro      | 0.22372  |
- Confrontos grandes com maior desequilíbrio (diferença média de gols):
|       

### Calculo do ELO com os resultados de 2025

In [16]:
import pandas as pd
import math

# Parâmetros do Elo
ELO_INICIAL = 1500
K_BASE = 20
VANTAGEM_CASA = 0

# Clubes-foco (ajuste conforme seu critério)
clubes_foco = [
    'flamengo', 'fluminense', 'vasco', 'botafogo',
    'palmeiras', 'sao paulo', 'santos', 'corinthians',
    'atletico mineiro', 'cruzeiro', 'gremio', 'internacional'
]

# Carregar e normalizar dados
df = pd.read_csv('/kaggle/input/br-2025/campeonato-brasileiro-full.csv')
df['mandante'] = df['mandante'].str.strip().str.lower()
df['visitante'] = df['visitante'].str.strip().str.lower()
df['resultado'] = df['resultado'].str.strip().str.lower()

def calcular_vantagens(df):
    v = {}
    for t in pd.unique(df['mandante']):
        saldo = (df[df['mandante']==t]['gols_mandante'] - df[df['mandante']==t]['gols_visitante']).mean()
        v[t] = 60 * (saldo**0.8) if saldo > 0 else 0
    return v

def atualiza_elo(r_c, r_v, g_c, g_v, v_c):
    if g_c > g_v:
        res, diff = 1.0, g_c - g_v
    elif g_c < g_v:
        res, diff = 0.0, g_v - g_c
    else:
        res, diff = 0.5, 0
    k = K_BASE * math.log(diff + 1) if res != 0.5 else K_BASE
    e_c = 1 / (1 + 10 ** ((r_v - (r_c + v_c)) / 400))
    return r_c + k * (res - e_c), r_v + k * ((1 - res) - (1 - e_c))

# Calcula vantagens de mando
vant = calcular_vantagens(df)

# Inicializa Elo
elo = {}

# Processa cada jogo na ordem do arquivo
for _, r in df.iterrows():
    m, v = r['mandante'], r['visitante']
    em = elo.get(m, ELO_INICIAL)
    ev = elo.get(v, ELO_INICIAL)
    nc, nv = atualiza_elo(em, ev, r['gols_mandante'], r['gols_visitante'], vant.get(m, VANTAGEM_CASA))
    elo[m], elo[v] = nc, nv

# Seleciona os Elos finais dos clubes-foco
elos_foco = {club: elo.get(club, ELO_INICIAL) for club in clubes_foco}
elos_ordenado = dict(sorted(elos_foco.items(), key=lambda x: x[1], reverse=True))

# Mostra a tabela Markdown
import pandas as pd
df_elos = pd.DataFrame(list(elos_ordenado.items()), columns=['Clube', 'Elo Final'])
print(df_elos.to_markdown(index=False))


| Clube            |   Elo Final |
|:-----------------|------------:|
| flamengo         |     1716.98 |
| palmeiras        |     1693.1  |
| botafogo         |     1677.7  |
| internacional    |     1636.93 |
| corinthians      |     1618.24 |
| sao paulo        |     1572.64 |
| cruzeiro         |     1563.15 |
| fluminense       |     1560.01 |
| atletico mineiro |     1550.48 |
| vasco            |     1530.13 |
| gremio           |     1510.5  |
| santos           |     1472.62 |


### Utilizando o modelo hibrido com os resultasos de 2025 (ELO e rodadas) para gerar uma previsão para a rodada do dia 12/07.

In [17]:
import pandas as pd
import math

ELO_RATING_INICIAL = 1500
ELO_K_FACTOR_BASE = 30
ELO_VANTAGEM_CASA_PADRAO = 80
POISSON_MAX_GOLS = 8
ELO_INFLUENCE = 0.35

def calcular_vantagens_casa(df):
    vantagens = {}
    times = pd.unique(df['mandante'])
    for time in times:
        jogos_casa = df[df['mandante'] == time]
        saldo_de_gols_medio = (jogos_casa['gols_mandante'] - jogos_casa['gols_visitante']).mean()
        if saldo_de_gols_medio and saldo_de_gols_medio > 0:
            vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
        else:
            vantagem_rating = 0
        vantagens[time] = vantagem_rating
    return vantagens

def prever_partida_hibrido(time_casa, time_visitante, elo_ratings, forcas_poisson, medias_liga, vantagens_casa):
    ataque_casa = forcas_poisson[time_casa]['ataque_casa']
    defesa_visitante = forcas_poisson[time_visitante]['defesa_fora']
    gols_base_casa = ataque_casa * defesa_visitante * medias_liga['gols_casa']
    ataque_visitante = forcas_poisson[time_visitante]['ataque_fora']
    defesa_casa = forcas_poisson[time_casa]['defesa_casa']
    gols_base_visitante = ataque_visitante * defesa_casa * medias_liga['gols_fora']
    rating_casa = elo_ratings.get(time_casa, ELO_RATING_INICIAL)
    rating_visitante = elo_ratings.get(time_visitante, ELO_RATING_INICIAL)
    vantagem_time_casa = vantagens_casa.get(time_casa, ELO_VANTAGEM_CASA_PADRAO)
    elo_diff = (rating_casa + vantagem_time_casa) - rating_visitante
    fator_ajuste_casa = 1 + (elo_diff / 1000) * ELO_INFLUENCE
    fator_ajuste_visitante = 1 - (elo_diff / 1000) * ELO_INFLUENCE
    gols_finais_casa = gols_base_casa * max(0.1, fator_ajuste_casa)
    gols_finais_visitante = gols_base_visitante * max(0.1, fator_ajuste_visitante)
    prob_vitoria_casa, prob_empate, prob_vitoria_visitante = 0, 0, 0
    for g_c in range(POISSON_MAX_GOLS + 1):
        for g_v in range(POISSON_MAX_GOLS + 1):
            prob_placar = poisson(gols_finais_casa, g_c) * poisson(gols_finais_visitante, g_v)
            if g_c > g_v: prob_vitoria_casa += prob_placar
            elif g_c == g_v: prob_empate += prob_placar
            else: prob_vitoria_visitante += prob_placar
    total_prob = prob_vitoria_casa + prob_empate + prob_vitoria_visitante
    probs = [prob_vitoria_casa, prob_empate, prob_vitoria_visitante]
    idx_max = probs.index(max(probs))
    palpite = ['Mandante', 'Empate', 'Visitante'][idx_max]
    return {
        'Mandante': time_casa.title(),
        'Visitante': time_visitante.title(),
        'Elo Mandante': int(rating_casa),
        'Elo Visitante': int(rating_visitante),
        'Gols Esp. Mandante': round(gols_finais_casa,2),
        'Gols Esp. Visitante': round(gols_finais_visitante,2),
        'P(Mandante)%': f"{prob_vitoria_casa / total_prob * 100:.1f}",
        'P(Empate)%': f"{prob_empate / total_prob * 100:.1f}",
        'P(Visitante)%': f"{prob_vitoria_visitante / total_prob * 100:.1f}",
        'Palpite': palpite
    }

def main():
    df = pd.read_csv('/kaggle/input/br-2025/campeonato-brasileiro-full.csv')
    df['mandante'] = df['mandante'].str.strip().str.lower()
    df['visitante'] = df['visitante'].str.strip().str.lower()
    df['gols_mandante'] = df['gols_mandante'].astype(int)
    df['gols_visitante'] = df['gols_visitante'].astype(int)
    forcas_poisson, medias_liga = calcular_forcas_poisson(df)
    vantagens_casa = calcular_vantagens_casa(df)
    elo_ratings = {}
    for _, partida in df.iterrows():
        time_c, time_v = partida['mandante'], partida['visitante']
        placar_c, placar_v = partida['gols_mandante'], partida['gols_visitante']
        rating_c = elo_ratings.get(time_c, ELO_RATING_INICIAL)
        rating_v = elo_ratings.get(time_v, ELO_RATING_INICIAL)
        vantagem_c = vantagens_casa.get(time_c, ELO_VANTAGEM_CASA_PADRAO)
        novo_rating_c, novo_rating_v = atualizar_ratings_elo(rating_c, rating_v, placar_c, placar_v, vantagem_c)
        elo_ratings[time_c], elo_ratings[time_v] = novo_rating_c, novo_rating_v

    jogos = [
        ('internacional','vitoria'),
        ('bahia','atletico mineiro'),
        ('corinthians','bragantino'),
        ('cruzeiro','gremio'),
        ('fortaleza','ceara'),
        ('juventude','sport'),
        ('flamengo','sao paulo'),
        ('vasco','botafogo'),
        ('santos','palmeiras'),
        ('mirassol','fluminense')
    ]
    resultados = []
    for casa, visita in jogos:
        try:
            r = prever_partida_hibrido(casa, visita, elo_ratings, forcas_poisson, medias_liga, vantagens_casa)
            resultados.append(r)
        except Exception as e:
            print(f"Erro na previsão {casa} x {visita}: {e}")
    df_resultados = pd.DataFrame(resultados)
    print(df_resultados.to_markdown(index=False))

if __name__ == "__main__":
    main()


KeyError: 'mandante_Placar'

## Prevendo o título

In [19]:
"""
Módulo de previsão de partidas de futebol utilizando modelos híbridos Poisson + Elo.
Inclui funções para calcular forças de ataque/defesa, vantagens de casa, prever resultados e atualizar ratings Elo.
"""
import math
import pandas as pd

ELO_RATING_INICIAL = 1500
ELO_K_FACTOR_BASE = 30
ELO_VANTAGEM_CASA_PADRAO = 30
POISSON_MAX_GOLS = 8
ELO_INFLUENCE = 0.10

def poisson(lmbda, k):
    """
    Calcula a probabilidade de k ocorrências para uma média lmbda usando a distribuição de Poisson.
    """
    return math.exp(-lmbda) * (lmbda ** k) / math.factorial(k)

def calcular_forcas_poisson(df):
    """
    Calcula as forças de ataque e defesa (em casa e fora) para cada time usando a média de gols,
    com base no modelo de Poisson, e retorna também as médias da liga.
    Parâmetros:
        df (pd.DataFrame): DataFrame contendo os jogos, mandantes, visitantes e gols.
    Retorna:
        tuple: (dicionário de forças por time, dicionário de médias da liga)
    """
    times = pd.unique(df['mandante'].tolist() + df['visitante'].tolist())
    forcas = {}
    # Médias da liga
    media_gols_casa = df['gols_mandante'].mean()
    media_gols_fora = df['gols_visitante'].mean()
    medias_liga = {'gols_casa': media_gols_casa, 'gols_fora': media_gols_fora}
    # Forças de ataque/defesa para cada time
    for time in times:
        # Ataque em casa: gols marcados em casa / jogos em casa / média da liga
        jogos_casa = df[df['mandante'] == time]
        jogos_fora = df[df['visitante'] == time]
        ataque_casa = (jogos_casa['gols_mandante'].mean() / media_gols_casa) if not jogos_casa.empty and media_gols_casa > 0 else 1.0
        defesa_casa = (jogos_casa['gols_visitante'].mean() / media_gols_fora) if not jogos_casa.empty and media_gols_fora > 0 else 1.0
        ataque_fora = (jogos_fora['gols_visitante'].mean() / media_gols_fora) if not jogos_fora.empty and media_gols_fora > 0 else 1.0
        defesa_fora = (jogos_fora['gols_mandante'].mean() / media_gols_casa) if not jogos_fora.empty and media_gols_casa > 0 else 1.0
        forcas[time] = {
            'ataque_casa': ataque_casa,
            'defesa_casa': defesa_casa,
            'ataque_fora': ataque_fora,
            'defesa_fora': defesa_fora
        }
    return forcas, medias_liga

def calcular_vantagens_casa(df):
    """
    Calcula a vantagem de jogar em casa para cada time com base no saldo médio de gols como mandante.
    Parâmetros:
        df (pd.DataFrame): DataFrame contendo os jogos, mandantes, visitantes e gols.
    Retorna:
        dict: Dicionário com a vantagem de casa (em pontos Elo) para cada time.
    """
    vantagens = {}
    times = pd.unique(df['mandante'])
    for time in times:
        jogos_casa = df[df['mandante'] == time]
        saldo_de_gols_medio = (jogos_casa['gols_mandante'] - jogos_casa['gols_visitante']).mean()
        if saldo_de_gols_medio and saldo_de_gols_medio > 0:
            vantagem_rating = 60 * (saldo_de_gols_medio ** 0.8)
        else:
            vantagem_rating = 0
        vantagens[time] = vantagem_rating
    return vantagens

def prever_partida_hibrido(time_casa, time_visitante, context):
    """
    Prevê o resultado de uma partida usando um modelo híbrido Poisson + Elo.

    Parâmetros:
        time_casa (str): Nome do time mandante.
        time_visitante (str): Nome do time visitante.
        context (dict): Dicionário com as chaves:
            - 'elo_ratings': dict com ratings Elo dos times
            - 'forcas_poisson': dict com forças de ataque/defesa dos times
            - 'medias_liga': dict com médias de gols da liga
            - 'vantagens_casa': dict com vantagens de casa dos times

    Retorna:
        dict: Resultados previstos da partida.
    """
    elo_ratings = context['elo_ratings']
    forcas_poisson = context['forcas_poisson']
    medias_liga = context['medias_liga']
    vantagens_casa = context['vantagens_casa']

    ataque_casa = forcas_poisson[time_casa]['ataque_casa']
    defesa_visitante = forcas_poisson[time_visitante]['defesa_fora']
    gols_base_casa = ataque_casa * defesa_visitante * medias_liga['gols_casa']
    ataque_visitante = forcas_poisson[time_visitante]['ataque_fora']
    defesa_casa = forcas_poisson[time_casa]['defesa_casa']
    gols_base_visitante = ataque_visitante * defesa_casa * medias_liga['gols_fora']
    rating_casa = elo_ratings.get(time_casa, ELO_RATING_INICIAL)
    rating_visitante = elo_ratings.get(time_visitante, ELO_RATING_INICIAL)
    vantagem_time_casa = vantagens_casa.get(time_casa, ELO_VANTAGEM_CASA_PADRAO)
    elo_diff = (rating_casa + vantagem_time_casa) - rating_visitante
    fator_ajuste_casa = 1 + (elo_diff / 1000) * ELO_INFLUENCE
    fator_ajuste_visitante = 1 - (elo_diff / 1000) * ELO_INFLUENCE
    gols_finais_casa = gols_base_casa * max(0.1, fator_ajuste_casa)
    gols_finais_visitante = gols_base_visitante * max(0.1, fator_ajuste_visitante)

    prob_vitoria_casa = 0
    prob_empate = 0
    prob_vitoria_visitante = 0
    for g_c in range(POISSON_MAX_GOLS + 1):
        for g_v in range(POISSON_MAX_GOLS + 1):
            prob_placar = poisson(gols_finais_casa, g_c) * poisson(gols_finais_visitante, g_v)
            if g_c > g_v:
                prob_vitoria_casa += prob_placar
            elif g_c == g_v:
                prob_empate += prob_placar
            else:
                prob_vitoria_visitante += prob_placar

    total_prob = prob_vitoria_casa + prob_empate + prob_vitoria_visitante
    probs = [prob_vitoria_casa, prob_empate, prob_vitoria_visitante]
    idx_max = probs.index(max(probs))
    palpite = ['Mandante', 'Empate', 'Visitante'][idx_max]
    return {
        'Mandante': time_casa.title(),
        'Visitante': time_visitante.title(),
        'Elo Mandante': int(rating_casa),
        'Elo Visitante': int(rating_visitante),
        'Gols Esp. Mandante': round(gols_finais_casa, 2),
        'Gols Esp. Visitante': round(gols_finais_visitante, 2),
        'P(Mandante)%': f"{prob_vitoria_casa / total_prob * 100:.1f}",
        'P(Empate)%': f"{prob_empate / total_prob * 100:.1f}",
        'P(Visitante)%': f"{prob_vitoria_visitante / total_prob * 100:.1f}",
        'Palpite': palpite
    }

def atualizar_ratings_elo(rating_c, rating_v, placar_c, placar_v, vantagem_c):
    """
    Atualiza os ratings Elo dos times mandante e visitante após uma partida.
    rating_c: Elo do mandante antes do jogo
    rating_v: Elo do visitante antes do jogo
    placar_c: Gols do mandante
    placar_v: Gols do visitante
    vantagem_c: Vantagem de casa (em pontos Elo)
    """
    # Resultado real
    if placar_c > placar_v:
        resultado_real = 1.0
    elif placar_c == placar_v:
        resultado_real = 0.5
    else:
        resultado_real = 0.0

    # Expectativa de resultado
    exp_c = 1 / (1 + 10 ** ((rating_v - (rating_c + vantagem_c)) / 400))
    exp_v = 1 - exp_c

    # K pode ser ajustado conforme a diferença de gols
    diff_gols = abs(placar_c - placar_v)
    k = ELO_K_FACTOR_BASE * (1 + 0.5 * max(0, diff_gols - 1))

    novo_rating_c = rating_c + k * (resultado_real - exp_c)
    novo_rating_v = rating_v + k * ((1 - resultado_real) - exp_v)
    return novo_rating_c, novo_rating_v

def main():
    df = pd.read_csv('/kaggle/input/br-2025/campeonato-brasileiro-full.csv')
    df['mandante'] = df['mandante'].str.strip().str.lower()
    df['visitante'] = df['visitante'].str.strip().str.lower()
    df['gols_mandante'] = df['gols_mandante'].astype(int)
    df['gols_visitante'] = df['gols_visitante'].astype(int)
    forcas_poisson, medias_liga = calcular_forcas_poisson(df)
    vantagens_casa = calcular_vantagens_casa(df)
    elo_ratings = {}
    for _, partida in df.iterrows():
        time_c, time_v = partida['mandante'], partida['visitante']
        placar_c, placar_v = partida['gols_mandante'], partida['gols_visitante']
        rating_c = elo_ratings.get(time_c, ELO_RATING_INICIAL)
        rating_v = elo_ratings.get(time_v, ELO_RATING_INICIAL)
        vantagem_c = vantagens_casa.get(time_c, ELO_VANTAGEM_CASA_PADRAO)
        novo_rating_c, novo_rating_v = atualizar_ratings_elo(rating_c, rating_v, placar_c, placar_v, vantagem_c)
        elo_ratings[time_c], elo_ratings[time_v] = novo_rating_c, novo_rating_v

    jogos = [
        ('internacional','vitoria'),
        ('bahia','atletico mineiro'),
        ('corinthians','bragantino'),
        ('cruzeiro','gremio'),
        ('fortaleza','ceara'),
        ('juventude','sport'),
        ('flamengo','sao paulo'),
        ('vasco','botafogo'),
        ('santos','palmeiras'),
        ('mirassol','fluminense')
    ]
    resultados = []
    context = {
        'elo_ratings': elo_ratings,
        'forcas_poisson': forcas_poisson,
        'medias_liga': medias_liga,
        'vantagens_casa': vantagens_casa
    }
    for casa, visita in jogos:
        try:
            r = prever_partida_hibrido(casa, visita, context)
            resultados.append(r)
        except Exception as e:
            print(f"Erro na previsão {casa} x {visita}: {e}")
    df_resultados = pd.DataFrame(resultados)

    # Renomear e reordenar colunas para melhor apresentação em markdown
    df_resultados = df_resultados.rename(columns={
        'Mandante': 'Mandante',
        'Visitante': 'Visitante',
        'Gols Esp. Mandante': 'Gols Mandante',
        'Gols Esp. Visitante': 'Gols Visitante',
        'P(Mandante)%': 'Prob Mandante (%)',
        'P(Empate)%': 'Prob Empate (%)',
        'P(Visitante)%': 'Prob Visitante (%)',
        'Palpite': 'Palpite'
    })
    colunas = [
        'Mandante', 'Visitante',
        'Gols Mandante', 'Gols Visitante',
        'Prob Mandante (%)', 'Prob Empate (%)', 'Prob Visitante (%)',
        'Palpite'
    ]
    # Garantir que as colunas existem
    colunas = [c for c in colunas if c in df_resultados.columns]
    df_resultados = df_resultados[colunas]

    # Arredondar valores numéricos
    df_resultados['Gols Mandante'] = df_resultados['Gols Mandante'].astype(float).round(2)
    df_resultados['Gols Visitante'] = df_resultados['Gols Visitante'].astype(float).round(2)
    df_resultados['Prob Mandante (%)'] = df_resultados['Prob Mandante (%)'].astype(float).round(1)
    df_resultados['Prob Empate (%)'] = df_resultados['Prob Empate (%)'].astype(float).round(1)
    df_resultados['Prob Visitante (%)'] = df_resultados['Prob Visitante (%)'].astype(float).round(1)

    # Adicionar título antes da tabela
    print("# Previsão de Jogos - Modelo Híbrido Poisson + Elo\n")
    print(df_resultados.to_markdown(index=False))

if __name__ == "__main__":
    main()


/tmp/ipykernel_35/779939709.py:29: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  times = pd.unique(df['mandante'].tolist() + df['visitante'].tolist())


# Previsão de Jogos - Modelo Híbrido Poisson + Elo

| Mandante      | Visitante        |   Gols Mandante |   Gols Visitante |   Prob Mandante (%) |   Prob Empate (%) |   Prob Visitante (%) | Palpite   |
|:--------------|:-----------------|----------------:|-----------------:|--------------------:|------------------:|---------------------:|:----------|
| Internacional | Vitoria          |            1.82 |             0.76 |                62.6 |              22.3 |                 15.1 | Mandante  |
| Bahia         | Atletico Mineiro |            1.28 |             1.18 |                38.8 |              27.2 |                 34   | Mandante  |
| Corinthians   | Bragantino       |            1.39 |             0.95 |                47.1 |              27.3 |                 25.6 | Mandante  |
| Cruzeiro      | Gremio           |            1.55 |             1.02 |                49.5 |              25.5 |                 25   | Mandante  |
| Fortaleza     | Ceara            |      

## Prevendo o resultado final com 10 mil simulações

In [ ]:
import pandas as pd
import numpy as np
import time
import os

import importlib.util

def import_previsao():
    spec = importlib.util.spec_from_file_location("previsao", "/kaggle/input/previsao-br-25/other/previsao/1/previsao.py")
    previsao = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(previsao)
    return previsao

previsao = import_previsao()
prever_partida_hibrido = previsao.prever_partida_hibrido
calcular_forcas_poisson = previsao.calcular_forcas_poisson
calcular_vantagens_casa = previsao.calcular_vantagens_casa
ELO_RATING_INICIAL = previsao.ELO_RATING_INICIAL

def normalizar_nome(time):
    nome = time.strip().lower()
    nome = (
        nome.replace("á", "a")
        .replace("ã", "a")
        .replace("â", "a")
        .replace("é", "e")
        .replace("ê", "e")
        .replace("í", "i")
        .replace("ó", "o")
        .replace("ô", "o")
        .replace("ú", "u")
        .replace("ç", "c")
    )
    mapa = {
        "cam": "atletico mineiro",
        "gremio": "gremio",
        "santo": "santos",
        "vitoria": "vitoria",
        "palmeiras": "palmeiras",
        "flamengo": "flamengo",
        "cruzeiro": "cruzeiro",
        "mirassol": "mirassol",
        "bahia": "bahia",
        "fluminense": "fluminense",
        "atletico mineiro": "atletico mineiro",
        "botafogo": "botafogo",
        "corinthians": "corinthians",
        "ceara": "ceara",
        "vasco": "vasco",
        "sao paulo": "sao paulo",
        "santos": "santos",
        "internacional": "internacional",
        "fortaleza": "fortaleza",
        "juventude": "juventude",
        "sport": "sport",
        "grêmio": "gremio",
    }
    return mapa.get(nome, nome)

def ler_classificacao_inicial(path="/kaggle/input/classificacao-10-rodada/classificacao.txt"):
    classificacao = {}
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line or "," not in line:
                continue
            nome, pontos = line.split(",", 1)
            nome = normalizar_nome(nome)
            try:
                pontos = int(pontos)
            except ValueError:
                pontos = 0
            classificacao[nome] = pontos
    return classificacao

def simular_campeonato(n_simulacoes=10000):
    # Carregar jogos futuros
    df_jogos = pd.read_csv("/kaggle/input/br-2025/campeonato-brasileiro-full.csv")
    df_jogos["mandante"] = df_jogos["mandante"].apply(normalizar_nome)
    df_jogos["visitante"] = df_jogos["visitante"].apply(normalizar_nome)

    # Carregar histórico para calcular forças/elos
    df_historico = pd.read_csv("/kaggle/input/br-2025/campeonato-brasileiro-full.csv")
    df_historico["mandante"] = df_historico["mandante"].apply(normalizar_nome)
    df_historico["visitante"] = df_historico["visitante"].apply(normalizar_nome)
    df_historico["gols_mandante"] = df_historico["gols_mandante"].astype(int)
    df_historico["gols_visitante"] = df_historico["gols_visitante"].astype(int)

    forcas_poisson, medias_liga = calcular_forcas_poisson(df_historico)
    vantagens_casa = calcular_vantagens_casa(df_historico)
    classif_inicial_norm = ler_classificacao_inicial()
    elo_ratings = {time: ELO_RATING_INICIAL for time in classif_inicial_norm.keys()}
    campeoes = {time: 0 for time in classif_inicial_norm.keys()}
    g4 = {time: 0 for time in classif_inicial_norm.keys()}
    g6 = {time: 0 for time in classif_inicial_norm.keys()}
    g8 = {time: 0 for time in classif_inicial_norm.keys()}
    z4 = {time: 0 for time in classif_inicial_norm.keys()}
    jogos_futuros = df_jogos[["mandante", "visitante"]].values.tolist()

    for sim in range(n_simulacoes):
        pontos = classif_inicial_norm.copy()
        for mandante, visitante in jogos_futuros:
            mandante = normalizar_nome(mandante)
            visitante = normalizar_nome(visitante)
            context = {
                "elo_ratings": elo_ratings,
                "forcas_poisson": forcas_poisson,
                "medias_liga": medias_liga,
                "vantagens_casa": vantagens_casa
            }
            try:
                previsao = prever_partida_hibrido(mandante, visitante, context)
                p_mandante = float(previsao["P(Mandante)%"])
                p_empate = float(previsao["P(Empate)%"])
                p_visitante = float(previsao["P(Visitante)%"])
                sorteio = np.random.rand() * 100
                if sorteio < p_mandante:
                    pontos[mandante] += 3
                elif sorteio < p_mandante + p_empate:
                    pontos[mandante] += 1
                    pontos[visitante] += 1
                else:
                    pontos[visitante] += 3
            except Exception:
                if mandante in pontos and visitante in pontos:
                    pontos[mandante] += 1
                    pontos[visitante] += 1
        max_pontos = max(pontos.values())
        campeoes_sim = [t for t, p in pontos.items() if p == max_pontos]
        for t in campeoes_sim:
            campeoes[t] += 1 / len(campeoes_sim)
        ranking = sorted(pontos.items(), key=lambda x: (-x[1], np.random.rand()))
        top4 = [t for t, _ in ranking[:4]]
        top6 = [t for t, _ in ranking[:6]]
        top8 = [t for t, _ in ranking[:8]]
        bot4 = [t for t, _ in ranking[-4:]]
        for t in top4:
            g4[t] += 1
        for t in top6:
            g6[t] += 1
        for t in top8:
            g8[t] += 1
        for t in bot4:
            z4[t] += 1

    print("# Probabilidades após simulação")
    total = n_simulacoes
    tabela = []
    for t in classif_inicial_norm.keys():
        pct_camp = campeoes[t] / total * 100
        pct_g4 = g4[t] / total * 100
        pct_g6 = g6[t] / total * 100
        pct_g8 = g8[t] / total * 100
        pct_z4 = z4[t] / total * 100
        tabela.append((t, pct_camp, pct_g4, pct_g6, pct_g8, pct_z4))
    tabela = sorted(tabela, key=lambda x: -x[1])  # Ordena por % campeão

    print("| # | Time | % Campeão | % G4 | % G6 | % G8 | % Z4 |")
    print("|---|------|-----------|------|------|------|------|")
    for idx, (time, pct_camp, pct_g4, pct_g6, pct_g8, pct_z4) in enumerate(tabela, 1):
        print(f"| {idx} | {time.title()} | {pct_camp:.2f}% | {pct_g4:.2f}% | {pct_g6:.2f}% | {pct_g8:.2f}% | {pct_z4:.2f}% |")

if __name__ == "__main__":
    inicio = time.time()
    simular_campeonato(10000)
    print(f"Tempo: {time.time() - inicio:.2f} segundos")

/kaggle/input/previsao-br-25/other/previsao/1/previsao.py:29: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  times = pd.unique(df['mandante'].tolist() + df['visitante'].tolist())


Analisando a 13 rodada do compeonato brasileiro, o meu modelo, sassamaru-25, acertou 5 de 7 resultados (não tivemos a rodada completa ainda), ou seja, **uma taxa de acerto de 71.83% por enquanto**.

Precisarei esperar o final da rodada para reajustar os rakings de elo dos times e de elenco. Mas, com **100 mil simulações** a probabilidade começa a ficar mais próxima da realidade.

Um resumo do que ocorreu (faltando 1 jogos e com outros dois adiados):

| Jogo                    | Sassamaru-25  | Realidade     |
|-------------------------|---------------|---------------|
| Flamengo x São Paulo    | **Flamengo**      | Flamengo      |
| Internacional x Vitória | **Internacional** | Internacional |
| Vasco x Botafogo        | Vasco         | Botafogo      |
| Bahia x Atletico MG     | **Bahia**         | Bahia         |
| Corinthians x Bragantino  | **Corinthians**   | Corinthians   |
| Fortaleza x Ceará       | Fortaleza     | Ceará         |
| Cruzeiro x Grêmio       | **Cruzeiro**      | Cruzeiro      |
| Juventude x Sport       | **Juventude**     | Juventude      |

Em negrito os acertos do modelo.

> **Update do dia 14/07**: como resultado de 2 x 0 pro Juventude contra o Sport, o modelo acertou 6 de 8 jogos. A porcentagem de acerto passou a ser de **75%**.